In [ ]:

decision_mode = '''Given the company's environment, the most appropriate production planning models from the book "Factory Physics" would be:

1. **Aggregate Planning (AP), Product Mix Planning II. Basic model**: This model is suitable because the company deals with one main product and multiple minor products. The basic product mix planning model will help in managing the production of multiple products explicitly.

2. **Aggregate Planning (AP), Product Mix Planning III. Capacitated resources**: Since the capacity of workstations, plant size, and labor force size are constraining factors, this model will help in managing these constraints effectively.

3. **Aggregate Planning (AP), Product Mix Planning VI. Overtime**: Given that the company uses overtime, this model will help in planning and optimizing the use of overtime to meet production demands.

4. **Workforce Planning (WP), VIII. Basic model**: This model is essential because the company has a mix of long-term and short-term contracts and uses overtime. Workforce planning will help in managing the labor force size and scheduling overtime efficiently.

These models combined will provide a comprehensive approach to managing the production planning, considering the constraints and variations in the company's environment.'''

Input_Data= '''
time_horizon = 6
products_total = 4
workstations_total = 4
ressources_total = 4
profit = [132.0, 813.0, 225.0, 131.0]
holding_costs = [13.0, 18.0, 15.0, 13.0]
min_sales = [
    [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
    [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
    [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
    [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
]
max_demand = [
    [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
    [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
    [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
    [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
]
capacity = [
    [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
    [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
    [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
    [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
]
production_time = [
    [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
    [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
    [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
    [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
]
resource_requirement = [
    [0.8, 0.5, 1.0, 4.0],
    [0.5, 1.3, 3.0, 1.0],
    [2.0, 1.0, 2.0, 4.0],
    [2.0, 6.0, 4.0, 2.0],
]
resource_capacity = [
    [7071.0, 29208.0, 11107.0, 4708.0, 2450.0, 2900.0],
    [4425.0, 22041.0, 3861.0, 2101.0, 14025.0, 2900.0],
    [70103.0, 1103.0, 20517.0, 4051.0, 9025.0, 1403.0],
    [200.0, 31601.0, 4709.0, 25019.0, 14500.0, 2003.0],
]
initial_inventory = [41.0, 58.0, 31.0, 38.0]
yield_loss = [
    [0.9, 0.7, 0.9, 0.8],
    [0.8, 0.6, 0.95, 0.8],
    [0.9, 0.75, 0.9, 0.78],
    [0.95, 0.87, 0.97, 0.98],
]
capacity_reduction_factor = 0.9


# New data required for the function
worker_hours_per_product = 2.0  # Assumption
hourly_wage = 20.0  # Assumption
hourly_wage_overtime = 35.0  # Assumption
hiring_cost = 10.0  # Assumption
layoff_cost = 5.0  # Assumption
initial_workers = 10  # Assumption


'''

# Defining some functions


def extract_python_code_manual(text):
    """
    Manually extracts Python code enclosed between ```python and ``` markers.

    Parameters:
    - text (str): The input text containing the Python code blocks.

    Returns:
    - list of str: A list containing all extracted Python code blocks.
    """
    code_blocks = []
    capture = False
    current_block = []

    # Split the text into lines for line-by-line processing
    lines = text.split('\n')

    for line in lines:
        # Check if the line contains the start marker
        if line.strip() == '```python':
            capture = True
            current_block = []  # Start a new block
        elif line.strip() == '```' and capture:
            # If we find the end marker and are in capture mode, stop capturing
            capture = False
            # Add the current block to the list of code blocks
            code_blocks.append('\n'.join(current_block))
        elif capture:
            # If we are in capture mode, add the line to the current block
            current_block.append(line)

    return code_blocks


def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    # Regular expression to find "Total Profit" followed by any whitespace and a number
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"

    # Search for the pattern in both strings
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)

    # Extract the profit values if found, otherwise default to 0
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0

    # Determine the higher profit and corresponding input string
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb

    # Return the higher profit value and the corresponding input string
    return winner_profit, winner_input, winner_fb

In [ ]:
!pip install simpy

In [ ]:
import numpy as np
import simpy
import random

def run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed=42):
    random.seed(seed)
    np.random.seed(seed)

    num_periods = 24  # Extended time periods
    back_orders_penalty = [5, 6, 7, 2]
    initial_inventory = [41.0, 58.0, 31.0, 38.0]

    # Specific profits and holding costs for each product
    profit_per_unit = [132.0, 83.0, 225.0, 131.0]
    holding_cost_per_unit = [3.0, 8.0, 5.0, 1.0]

    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    # Generate random data for extended periods
    def generate_extended_data(base_data):
        extended_data = []
        for row in base_data:
            extended_row = row + [random.uniform(min(row), max(row)) for _ in range(num_periods - len(row))]
            extended_data.append(extended_row)
        return extended_data

    capacity_reduction_factor = 0.9
    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]

    Workstation_capacities = [25, 75, 30]

    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]

    product_workflows = [
        [0, 1],  # Product 0
        [1, 2],  # Product 1
        [0],     # Product 2
        [1, 2],  # Product 3
    ]

    env = simpy.Environment()

    class Workstation:
        def __init__(self, env, capacity, id):
            self.env = env
            self.capacity = capacity
            self.id = id
            self.queue = simpy.Resource(env, capacity=capacity)
            self.working = True

        def fail(self, duration):
            self.working = False
            yield self.env.timeout(duration)
            self.working = True

        def process(self, produced):
            if self.working:
                with self.queue.request() as request:
                    yield request
                    processing_time = produced / self.capacity
                    yield self.env.timeout(processing_time)
            else:
                yield self.env.timeout(1)  # Wait for 1 unit of time before trying again

    class WorkerShift:
        def __init__(self, env, shifts):
            self.env = env
            self.shifts = shifts
            self.current_shift = 0
            self.current_workers = shifts[0]['workers']

        def change_shift(self):
            while True:
                yield self.env.timeout(8)  # Change shift every 8 hours
                self.current_shift = (self.current_shift + 1) % len(self.shifts)
                self.current_workers = self.shifts[self.current_shift]['workers']
                print(f"Shift change at time {self.env.now:.2f}. Current workers: {self.current_workers}")

    class MultiProductFactory:
        def __init__(self, env, workstations, worker_shift):
            self.env = env
            self.inventory = list(initial_inventory)
            self.total_profits = 0.0
            self.period_results = []
            self.workstations = workstations
            self.worker_shift = worker_shift
            self.resources = [list(cap) for cap in capacity]  # Initialize resource capacities

        def process_at_workstation(self, workstation, produced, product_id):
            with workstation.queue.request() as request:
                yield request
                processing_time = produced / workstation.capacity
                yield self.env.timeout(processing_time)

        def run_factory(self):
            for period in range(num_periods):
                print(f"\n--- Period {period + 1} ---")
                period_data = {'Period': period, 'Products': []}
                total_produced = 0
                total_sales = 0

                # Simulate workstation outages randomly
                for ws in self.workstations:
                    if ws.id == 2:  # Example: Specific workstation id for potential outage
                        if np.random.poisson(1/5) >= 1:  # Poisson distribution to simulate outages
                            downtime_duration = 2  # Downtime duration in time units
                            print(f"Workstation {ws.id} starting downtime at time {self.env.now} for {downtime_duration} periods.")
                            yield self.env.process(ws.fail(downtime_duration))
                            print(f"Workstation {ws.id} ending downtime at time {self.env.now}.")

                for product_id in range(len(self.inventory)):
                    produced = produced_Prev[product_id][period]

                    # Resource requirements and checking capacities
                    for workstation_id in product_workflows[product_id]:
                        workstation = self.workstations[workstation_id]
                        required_resource = produced * production_time[workstation_id][period] / yield_loss[workstation_id][product_id]
                        if required_resource > self.resources[workstation_id][period]:
                            produced = self.resources[workstation_id][period] * yield_loss[workstation_id][product_id] / production_time[workstation_id][period]
                            self.resources[workstation_id][period] = 0
                        else:
                            self.resources[workstation_id][period] -= required_resource

                    self.inventory[product_id] += produced
                    total_produced += produced

                    if produced > 0:
                        yield self.env.process(self.process_at_workstation(workstation, produced, product_id))

                    fluctuation = random.uniform(-0.1, 0.1)
                    actual_demand = sale_Prev[product_id][period] * (1 + fluctuation)
                    actual_demand = min(actual_demand, max_demand[product_id][period])
                    sold = max(min(self.inventory[product_id], actual_demand), min_sales[product_id][period])
                    missed_sales = max(actual_demand - sold, 0)

                    revenue = sold * profit_per_unit[product_id]
                    holding_costs = self.inventory[product_id] * holding_cost_per_unit[product_id]
                    period_profit = revenue - holding_costs
                    self.total_profits += period_profit

                    self.inventory[product_id] = max(self.inventory[product_id] - sold, 0)

                    product_data = {
                        'Product ID': product_id,
                        'Produced': produced,
                        'Sold': sold,
                        'Missed Sales': missed_sales,
                        'Ending Inventory': self.inventory[product_id],
                        'Period Profit': period_profit
                    }
                    period_data['Products'].append(product_data)

                total_required_worker_hours = total_produced * worker_hours_per_product
                overtime_hours = max(total_required_worker_hours - self.worker_shift.current_workers * 8, 0)
                regular_hours = min(total_required_worker_hours, self.worker_shift.current_workers * 8)
                hiring_needed = max((total_required_worker_hours - regular_hours - overtime_hours) / 8, 0)
                layoffs_needed = max((self.worker_shift.current_workers * 8 - total_required_worker_hours) / 8, 0)

                self.worker_shift.current_workers += hiring_needed - layoffs_needed

                # Calculate workforce utilization
                available_worker_hours = self.worker_shift.current_workers * 8  # Assuming each worker works for 8 hours per shift
                workforce_utilization = total_required_worker_hours / available_worker_hours if available_worker_hours > 0 else 0

                overtime_cost = overtime_hours * hourly_wage_overtime
                regular_cost = regular_hours * hourly_wage
                hiring_cost_total = hiring_needed * hiring_cost
                layoff_cost_total = layoffs_needed * layoff_cost

                period_profit -= (overtime_cost + regular_cost + hiring_cost_total + layoff_cost_total)
                self.total_profits += period_profit

                period_data.update({
                    'Overtime Hours': overtime_hours,
                    'Regular Hours': regular_hours,
                    'Hired': hiring_needed,
                    'Fired': layoffs_needed,
                    'Workforce': self.worker_shift.current_workers,
                    'Workforce Utilization': workforce_utilization
                })

                self.period_results.append(period_data)
                yield self.env.timeout(1)

    produced_Prev = np.array(produced_Prev)
    sale_Prev = np.array(sale_Prev)
    min_sales = np.array(min_sales)

    workstations = [Workstation(env, capacity, id) for id, capacity in enumerate(Workstation_capacities)]
    worker_shift = WorkerShift(env, shifts=[
        {'start': 0, 'end': 8, 'workers': 223},
        {'start': 8, 'end': 16, 'workers': 230},
        {'start': 16, 'end': 24, 'workers': 55}
    ])
    factory = MultiProductFactory(env, workstations, worker_shift)

    env.process(worker_shift.change_shift())
    env.process(factory.run_factory())
    env.run(until=num_periods * 24)  # Ensure the environment runs through all periods

    return factory.period_results, f"\nTotal Profits across all products and periods: {factory.total_profits:.2f}"

In [ ]:
explain_example = """ # Table: New Strategies
# | Strategy | Description | Reason for Implementation |
# | --- | --- | --- |
# | [TEXT] | [TEXT] | [TEXT] |
"""

In [ ]:
# Define the filename
filename = "8 Code Pattern.txt"

# Read the content of the file
with open(filename, 'r') as file:
    RAG = file.read()

# Print the content
print(RAG)

type(RAG)

Function to optimize aggregate procution planning with a single product.

Parameters:
t_bar (int): Time horizon.
revenue (float): Revenue per unit.
holding_costs (float): Holding costs per unit.
max_demand (dict): Demand for each period.
capacity (dict): Capacity for each period.
initial_inventory (float): Initial inventory at the start of the period.

Returns:
dict: Optimal values of decision variables and total profit.
"""
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
def optimize_production(time_horizon, profit, holding_costs, max_demand, capacity, initial_inventory):


    # Index ranges for products, workstations, and periods
    # periods = range(1, bar_t + 1)

    # Define the problem
    problem = LpProblem("Maximize_Profit", LpMaximize)

    # Create decision variables
    Amount_produced = [LpVariable(f"Amount_produced_period:{t}", lowBound=0) for t in range(time_horizon)]
    Amount_sold = [LpVariable(f"Amount_sold_period:{t}", lowBound=0) for t in range(time_hor

str

In [ ]:
!pip install simpy

In [ ]:
import subprocess
import sys

def install_simpy():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "simpy"])

def install_numpy():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy"])

def install_pulp():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp"])

def install_openai():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openai"])

# Function to install all packages
def install_all_packages():
    install_simpy()
    install_numpy()
    install_pulp()
    install_openai()

# Install all packages
install_all_packages()


In [ ]:
import simpy
import random
import numpy as np
import io
import sys
import ast
import re
import pulp
import random
import os

In [ ]:
!pip install openai==0.28

import os
import openai

openai.api_key = ''
os.environ['OPENAI_API_KEY'] = openai.api_key

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.37.1
    Uninstalling openai-1.37.1:
      Successfully uninstalled openai-1.37.1


In [ ]:
Num_Data = Input_Data

# Construct the chat history with the current data and previous solution
chat_history = [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": f"Print out the Python code for the following mathematical problem: {decision_made}. Incorporate all variable costs into the python function of the mathematic model. This shall serve as a 8 code patterns pattern: {RAG}. Use one code pattern or a blend of multiple code patterns. Use as the follow data as real life input data and include it in the code outputted: {Num_Data}. Futher instruction: Only print executable code in Python, noting above and below, because I immediately want to execute it using the eval. Write the code in a way that when running the code, the dicision variables will be outputted as (multidimensional) arrays of naked numbers. Also return the profit earned. Therefore, the input data (and its dimensions) you choose to include into the code and the mathemtaic model must match the optimization model. For the decision varaibles amount produced and sales and stock use this exact terminology as printouts: Produced: XXX, Sale: XXX, Stock: XXX. In case the LP you create includes workforce planning, also include: Workforce: XXX, Hires: XXX, Firings: XXX, Overtime: XXX"}
]

response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
print(response.choices[0].message.content)

# Save the alternative solution in the secondary_solutions list
previous_solution = response.choices[0].message.content

# At this point, secondary_solutions contains the new solutions corresponding to each item in data_list

```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Define the optimization function
def advanced_supply_chain_optimization():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]
    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]
    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]
    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 200

In [ ]:
response.choices[0].message.content

NameError: name 'response' is not defined

In [ ]:
Schablone = """['Certainly! I\'ll correct the error by ensuring all necessary variables, such as `products_total` and `time_horizon`, are defined before being used. Additionally, I\'ll implement the new strategy using Mixed-Integer Non-Linear Programming (MINLP) while keeping your input data intact and ensuring the solution is executable.\n\nBelow is the updated Python code:\n\n```python\nimport pulp\nimport numpy as np\nfrom scipy.optimize import minimize\n\n# Input data\ntime_horizon = 24  # Periods\nproducts_total = 4\nworkstations_total = 4\nressources_total = 4\n\nprofit = [132.0, 813.0, 225.0, 131.0]\nholding_costs = [13.0, 18.0, 15.0, 13.0]\n\nmin_sales = [\n    [71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n    [5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n    [23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n    [20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n]\n\nmax_demand = [\n    [371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n    [425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n    [203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n    [200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n]\n\ncapacity = [\n    [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n    [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n    [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n    [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n]\n\nproduction_time = [\n    [1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n    [2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n    [2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n    [2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n]\n\ninitial_inventory = [41.0, 58.0, 31.0, 38.0]\nyield_loss = [\n    [0.9, 0.7, 0.9, 0.8],\n    [0.8, 0.6, 0.95, 0.8],\n    [0.9, 0.75, 0.9, 0.78],\n    [0.95, 0.87, 0.97, 0.98],\n]\ncapacity_reduction_factor = 0.9\n\nworker_hours_per_product = 2.0  # Assumption\nhourly_wage = 20.0  # Assumption\nhourly_wage_overtime = 35.0  # Assumption\nhiring_cost = 10.0  # Assumption\nlayoff_cost = 5.0  # Assumption\ninitial_workers = 10  # Assumption\n\ndef minlp_optimization_model(time_horizon, products_total, workstations_total, max_demand, min_sales, \n                             production_time, capacity, profit, holding_costs, initial_inventory, \n                             yield_loss, capacity_reduction_factor, worker_hours_per_product, \n                             hourly_wage, hourly_wage_overtime, hiring_cost, layoff_cost, initial_workers):\n    \n    # Create the problem\n    problem = pulp.LpProblem("Advanced_Supply_Chain_Optimization_MINLP", pulp.LpMaximize)\n\n    # Decision Variables\n    Produced = [[pulp.LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in range(time_horizon)] for i in range(products_total)]\n    Sale = [[pulp.LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in range(time_horizon)] for i in range(products_total)]\n    Stock = [[pulp.LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in range(time_horizon)] for i in range(products_total)]\n    Workforce = [pulp.LpVariable(f"Workforce_{t}", lowBound=0) for t in range(time_horizon)]\n    Hires = [pulp.LpVariable(f"Hires_{t}", lowBound=0) for t in range(time_horizon)]\n    Firings = [pulp.LpVariable(f"Firings_{t}", lowBound=0) for t in range(time_horizon)]\n    Overtime = [pulp.LpVariable(f"Overtime_{t}", lowBound=0) for t in range(time_horizon)]\n\n    # Objective Function\n    profit_term = pulp.lpSum(profit[i] * Sale[i][t] for i in range(products_total) for t in range(time_horizon))\n    holding_cost_term = pulp.lpSum(holding_costs[i] * Stock[i][t] for i in range(products_total) for t in range(time_horizon))\n    worker_cost_term = pulp.lpSum(hourly_wage * Workforce[t] for t in range(time_horizon))\n    overtime_cost_term = pulp.lpSum(hourly_wage_overtime * Overtime[t] for t in range(time_horizon))\n    hires_cost_term = pulp.lpSum(hiring_cost * Hires[t] for t in range(time_horizon))\n    firings_cost_term = pulp.lpSum(layoff_cost * Firings[t] for t in range(time_horizon))\n\n    # Adding a non-linear profit term to simulate economies of scale\n    scale_factor = 0.95  # Simulating economies of scale where more production reduces cost per unit\n    scale_profit_term = pulp.lpSum((profit[i] * Produced[i][t] - (production_time[i][j]**scale_factor))\n                                   for i in range(products_total)\n                                   for t in range(time_horizon)\n                                   for j in range(workstations_total))\n    \n    problem += profit_term + scale_profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in range(products_total):\n        for t in range(time_horizon):\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in range(workstations_total):\n        for t in range(time_horizon):\n            problem += pulp.lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j]\n                                  for i in range(products_total)) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    # New strategy: Limit hiring and firing in certain periods\n    min_hiring_period = [1 if t % 4 == 0 else 0 for t in range(time_horizon)]\n    min_firing_period = [1 if t % 4 == 2 else 0 for t in range(time_horizon)]\n\n    for t in range(time_horizon):\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n        problem += pulp.lpSum(Produced[i][t] for i in range(products_total)) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        \n        if min_hiring_period[t] == 1:\n            problem += Hires[t] <= initial_workers, f"Min_Hiring_Period_{t}"\n        if min_firing_period[t] == 1:\n            problem += Firings[t] <= initial_workers, f"Min_Firing_Period_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],\n        "Sale": [[Sale[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],\n        "Stock": [[Stock[i][t].varValue for t in range(time_horizon)] for i in range(products_total)],\n        "Workforce": [Workforce[t].varValue for t in range(time_horizon)],\n        "Hires": [Hires[t].varValue for t in range(time_horizon)],\n        "Firings": [Firings[t].varValue for t in range(time_horizon)],\n        "Overtime": [Overtime[t].varValue for t in range(time_horizon)],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Call the function to optimize production\nminlp_optimization_model(time_horizon, products_total, workstations_total, max_demand, min_sales, \n                         production_time, capacity, profit, holding_costs, initial_inventory, \n                         yield_loss, capacity_reduction_factor, worker_hours_per_product, \n                         hourly_wage, hourly_wage_overtime, hiring_cost, layoff_cost, initial_workers)\n```\n\n### Table: Implemented Strategies\n| Strategy | Description | Reason for Implementation |\n| --- | --- | --- |\n| Limit Hiring/Firing Period | Restrict hiring and firing to certain periods to stabilize the workforce and reduce costs associated with frequent changes. | Based on the performance feedback, frequent workforce changes were leading to high costs. This strategy aims to stabilize the workforce over the periods. |\n\n### Real Life Implications and Interpretation\nBy introducing workforce stabilization strategies, the manufacturing system will see fewer disruptions caused by constant hiring and firing. This approach would lead to better prediction and planning capabilities, leading to potentially smoother operations and less time-consuming workforce training and onboarding processes. In essence, the factory\'s workforce would become more stable and efficient, leading to a more predictable and potentially more productive environment.\n\nThe code now ensures all variables are properly defined and used within the function. Run the code to see the improved results with the new strategy in place.']."""

In [ ]:
previous_solution = """```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import random

def optimize_production():
    # Input data
    time_horizon = 24  # Extended time periods
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0, 371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0, 425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0, 203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0, 200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0, 3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0, 4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0, 203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0, 200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
    ]

    production_time = [
        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0, 1.0, 2.0, 1.0, 4.0, 2.0, 9.0],
        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0, 2.0, 1.0, 3.0, 1.0, 5.0, 9.0],
        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0, 2.0, 1.0, 2.0, 4.0, 5.0, 1.0],
        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0, 2.0, 6.0, 4.0, 2.0, 1.0, 2.0],
    ]


    initial_inventory = [41.0, 58.0, 31.0, 38.0]
    yield_loss = [
        [0.9, 0.7, 0.9, 0.8],
        [0.8, 0.6, 0.95, 0.8],
        [0.9, 0.75, 0.9, 0.78],
        [0.95, 0.87, 0.97, 0.98],
    ]
    capacity_reduction_factor = 0.9

    # New data required for the function
    worker_hours_per_product = 2.0  # Assumption
    hourly_wage = 20.0  # Assumption
    hourly_wage_overtime = 35.0  # Assumption
    hiring_cost = 10.0  # Assumption
    layoff_cost = 5.0  # Assumption
    initial_workers = 10  # Assumption

    # Create the problem
    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)

    # Index ranges for products, workstations, and periods
    products = range(products_total)
    workstations = range(workstations_total)
    periods = range(time_horizon)

    # Decision Variables
    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat='Continuous') for t in periods] for i in products]
    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]
    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]
    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]
    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]

    # Objective Function
    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)
    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)
    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)
    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)
    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)
    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)
    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term

    # Constraints
    for i in products:
        for t in periods:
            # Sales constraints
            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"
            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"
            # Inventory balance constraint
            if t == 0:
                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"
            else:
                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"

    for j in workstations:
        for t in periods:
            # Adjusted capacity constraint
            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \
                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"

    for t in periods:
        # Workforce constraints
        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"
        if t == 0:
            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"
        else:
            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"

    # Solve the problem
    problem.solve()

    # Output the results
    results = {
        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],
        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],
        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],
        "Workforce": [Workforce[t].varValue for t in periods],
        "Hires": [Hires[t].varValue for t in periods],
        "Firings": [Firings[t].varValue for t in periods],
        "Overtime": [Overtime[t].varValue for t in periods],
        "Profit": pulp.value(problem.objective)
    }

    for key, value in results.items():
        print(f"{key}: {value}")

    return results

# Call the function to optimize production
optimize_production()
```"""

In [ ]:
import random
seed = 42

py_code_p = extract_python_code_manual(previous_solution)
# Redirect stdout to capture prints
old_stdout = sys.stdout  # Save the old state
new_stdout = io.StringIO()
sys.stdout = new_stdout
# Execute the code
exec(py_code_p[0])

# Restore stdout to its original state
sys.stdout = old_stdout
# Get the output and work with it
execp = new_stdout.getvalue()


In [ ]:
execp

'Produced: [[30.0, 28.0, 110.0, 123.0, 0.0, 171.24692, 20.130306, 62.214029, 45.408742, 145.2207, 0.0, 171.24692, 20.130306, 35.143548, 50.258519, 123.0, 0.0, 143.6788, 20.130306, 135.1909, 0.0, 123.0, 0.0, 115.9908], [0.0, 0.0, 0.0, 189.23851, 47.761491, 55.633846, 0.0, 0.0, 0.0, 749.67724, 47.761491, 55.633846, 0.0, 0.0, 3.0, 783.20156, 47.761491, 102.89349, 0.0, 0.0, 169.38193, 318.23851, 47.761491, 90.0], [0.0, 78.34, 27.0, 158.82686, 11.173137, 171.0, 0.0, 0.0, 159.87055, 25.956311, 11.173137, 171.0, 0.0, 0.0, 145.72817, 40.09869, 11.173137, 171.0, 0.0, 0.0, 45.447592, 140.37927, 11.173137, 143.0], [27.0, 0.0, 0.0, 91.657013, 0.0, 0.0, 38.880751, 64.634818, 0.0, 27.827418, 0.0, 0.0, 38.880751, 78.922016, 0.0, 26.197233, 0.0, 0.0, 38.880751, 26.119249, 0.0, 59.0, 0.0, 20.0]]\nSale: [[71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 115.9908], [5.0, 21.0, 3.0, 211.0, 55.0, 26.633846, 5.0, 21

In [ ]:
#patterns = {
#    "Produced": r"Produced: (\[\[.*?\]\])",
#    "Sale": r"Sale: (\[\[.*?\]\])",
#    "Stock": r"Stock: (\[\[.*?\]\])"
#}

patterns = {
    "Produced": r"Produced: (\[\[.*?\]\])",
    "Sale": r"Sale: (\[\[.*?\]\])",
    "Stock": r"Stock: (\[\[.*?\]\])",
    "Overtime": r"Overtime: (\[.*?\])",
    "Workforce": r"Workforce: (\[.*?\])",
    "Hires": r"Hires: (\[.*?\])",
    "Firings": r"Firings: (\[.*?\])",
    "Profit": r"Profit: (.*?)\n"
}

# Dictionary to hold the extracted matrices
matrices = {}

# Extract and evaluate each matrix
for key, pattern in patterns.items():
    match = re.search(pattern, execp, re.DOTALL)
    if match:
        matrices[key] = ast.literal_eval(match.group(1))

# Access the matrices
produced_Prev = matrices.get("Produced")
sale_Prev = matrices.get("Sale")
stock_Prev = matrices.get("Stock")

overtime_Prev = matrices.get("Overtime")
workers_Prev = matrices.get("Workforce")
hired_Prev = matrices.get("Hires")
fired_Prev = matrices.get("Firings")

In [ ]:
previous_feedback = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed = 42)[-1]


--- Period 1 ---

--- Period 2 ---
Workstation 2 starting downtime at time 2.3 for 2 periods.
Workstation 2 ending downtime at time 4.3.
Shift change at time 8.00. Current workers: 230

--- Period 3 ---

--- Period 4 ---
Shift change at time 16.00. Current workers: 55
Shift change at time 24.00. Current workers: 223

--- Period 5 ---
Shift change at time 32.00. Current workers: 230

--- Period 6 ---

--- Period 7 ---
Workstation 2 starting downtime at time 34.38185064666667 for 2 periods.
Workstation 2 ending downtime at time 36.38185064666667.

--- Period 8 ---
Shift change at time 40.00. Current workers: 55

--- Period 9 ---
Shift change at time 48.00. Current workers: 223

--- Period 10 ---
Workstation 2 starting downtime at time 50.930565973333344 for 2 periods.
Workstation 2 ending downtime at time 52.930565973333344.
Shift change at time 56.00. Current workers: 230
Shift change at time 64.00. Current workers: 55
Shift change at time 72.00. Current workers: 223
Shift change at ti

In [ ]:
previous_solution_bu = previous_solution
previous_feedback_bu = previous_feedback

In [ ]:
!pip install scipy numpy

This is the real itertive process (down below)

In [ ]:
def cache_thing(name, thing):
    from datetime import datetime
    import os

    # Create the cache directory if it doesn't exist
    if not os.path.exists('cache'):
        os.makedirs('cache')
    with open(f'cache/{name}-{datetime.now()}.txt', 'w') as f:
        f.write(thing)

In [ ]:
import openai
import sys
import io
import re
import ast
import random


def chance(ps, pf, explain_example, number_of_executions = 2):
    previous_solution = ps
    previous_feedback = pf

    winner_input = previous_solution

    counter = 0
    seed = 42

    while counter < number_of_executions:
        chat_history = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"""Print out the Python MINLP code for a production planning problem. {previous_solution} is the current solution.
            Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code).
            In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints.
            Also use Mixed-Integer Non-Linear Programming (MINLP) for objective function and constraints using scipy package which I installed already (numpy and pulp installed already too).
            Also use embedded interpretable functions. Whenever you modify the optimization model, include to say what the real life implications would be, how this modification would be interpreted in a real life manufacturing system.
            Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}.
            To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution.
            Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods.
            Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays.
            Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... (don't forget to print out any decision variable that was also used in the initial optimization model code.
            Print it out as a python list not numpy list.) If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against.
            Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example} (collecting the stretegies in this table, not deleting the older ones).
            These are just examples and should show the data structure! Try out only one new additionally added extension/new strategy compared to the LP model that I've shown you as the current solution. In case you see a modification and its annotation already you can do another one.
            Modify the optimization model and make use of non-linear stretegies!, while making sure the model is executable as it is with pulp and scipy having been installed already. THis can serve you as an example what i mean: {Schablone}.
            Don't necessarily copy the exact strategies, its serves to show you sth. about the MINLP formatting (btw, in this example that you should iterate on, initial workers at just and integer, not an array, unlike the MINLP formatting pattern I've shown you). It can show you how to create an executable code. ONe last thing: Keep in mind that I've inputted 24 periods that resemble 4 week, 6 working days each. THis can also be considered when implementing further strategies
            The amount of time periods is 24 (24 columns)!. Every 6 periods are one working week. You can take this into account when designing strategies"""}
        ]

        attempt = 1
        gen_out = None
        while attempt <= 3:
            old_stdout = sys.stdout  # Save the old state
            try:
                response = openai.ChatCompletion.create(model="gpt-4o-mini-2024-07-18", messages=chat_history)
                print(response.choices[0].message.content)
                cache_thing('response', response.choices[0].message.content)

                prediction = response.choices[0].message.content
                secondary_solution = "".join(prediction)
                print(secondary_solution)

                # Extracting the python code
                py_code_s = extract_python_code_manual(secondary_solution)

                # Redirect stdout to capture prints
                new_stdout = io.StringIO()
                sys.stdout = new_stdout
                # Execute the code
                exec(py_code_s[0])

                # Access the matrices
                produced_Prev = matrices.get("Produced")
                # ... (rest of the code)

                # Before using produced_Prev in run_extended_simulation, check its shape
                print("Shape of produced_Prev:", len(produced_Prev), "x", len(produced_Prev[0]) if produced_Prev else 0)

                # Get the output and work with it
                gen_out = new_stdout.getvalue()
                cache_thing('eval', gen_out)

                # Restore stdout to its original state
                sys.stdout = old_stdout

                attempt += 1
                break
            except Exception as e:
                print(f'Failed attempt {attempt} due to error: {str(e)}')
                error_feedback = str(e)
                chat_history.append({"role": "user", "content": f"The execution failed with the following error: {error_feedback}. Modify the previous solution to correct this error and make sure the new solution runs successfully."})
                attempt += 1
            finally:
              sys.stdout = old_stdout

        if gen_out is None:
            print(f"Attempt {attempt} failed three times. Proceeding to the next iteration.")
            counter += 1
            continue

        py_code_p = extract_python_code_manual(previous_solution)
        cache_thing('py_code_p', str(py_code_p))

        # Redirect stdout to capture prints
        old_stdout = sys.stdout  # Save the old state
        new_stdout = io.StringIO()
        sys.stdout = new_stdout

        # Execute the code
        exec(py_code_p[0])
        # Restore stdout to its original state
        sys.stdout = old_stdout
        # Get the output and work with it
        execp = new_stdout.getvalue()

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        cache_thing('execp', execp)
        cache_thing('prev_matrices', str({ 'matrices': str(matrices), 'seed': seed }))

        # Access the matrices
        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, gen_out, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed)[1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, seed)[1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, seed)[0]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, seed)[0]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        # Increment the counter at the end of each loop iteration
        counter += 1
        seed = seed+1

    return winner_input


In [ ]:
import concurrent.futures
import time
import sys
import io
import re
import ast

In [ ]:
# Define a function to run parallel tasks, before, the setting was 5/3   max_executions=10
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=1, max_executions=1):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, max_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

In [ ]:
collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)

Sure! Below, I provide an updated version of your Python production planning model using Mixed-Integer Non-Linear Programming (MINLP) with embedded interpretable functions. This model includes adjustments to the optimization function and adds new constraints based on the feedback from the previous simulation performance. 

```python
import pulp
import numpy as np

# Input data
time_horizon = 24  # Extended time periods
products_total = 4
workstations_total = 4

profit = [132.0, 813.0, 225.0, 131.0]
holding_costs = [13.0, 18.0, 15.0, 13.0]

min_sales = [
    [71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
    [5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
    [23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.

In [ ]:
collect_array

['```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\nimport random\n\ndef optimize_production():\n    # Input data\n    time_horizon = 24  # Extended time periods\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0, 71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0, 5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0, 23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0, 20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n  

In [ ]:
collect_array = ['\n\nHere is the modified code with new strategies for storage, production, and sales:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # New strategy for storage: Implement a safety stock policy for products 1 and 2 in periods 2 and 3\n    for i in [1, 2]:\n        for t in [2, 3]:\n            problem += Stock[i][t] >= 0.2 * max_demand[i][t], f"Safety_Stock_{i}_{t}"\n\n    # New strategy for production: Implement a production smoothing policy for product 3 in periods 1 and 2\n    for t in [1, 2]:\n        problem += Produced[3][t] >= 0.8 * Produced[3][t-1], f"Production_Smoothing_{t}"\n\n    # New strategy for sales: Implement a price discount policy for product 0 in period 4\n    problem += Sale[0][4] >= 1.1 * Sale[0][3], f"Price_Discount_4"\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Safety Stock | Implement a safety stock policy for products 1 and 2 in periods 2 and 3 | To mitigate stockouts and ensure sufficient inventory levels |\n# | Production Smoothing | Implement a production smoothing policy for product 3 in periods 1 and 2 | To reduce production variability and improve production planning |\n# | Price Discount | Implement a price discount policy for product 0 in period 4 | To increase sales and revenue |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added three new strategies:\n\n1. **Safety Stock**: Implement a safety stock policy for products 1 and 2 in periods 2 and 3 to mitigate stockouts and ensure sufficient inventory levels.\n2. **Production Smoothing**: Implement a production smoothing policy for product 3 in periods 1 and 2 to reduce production variability and improve production planning.\n3. **Price Discount**: Implement a price discount policy for product 0 in period 4 to increase sales and revenue.\n\nThese strategies are implemented through additional constraints and objective function terms. The `print_result` function is used to output the decision variables and profit earned. The table at the end summarizes the new strategies, their descriptions, and reasons for implementation.',
 '\n\nHere is the modified code with new strategies added:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # New data required for the function\n    storage_cost = 5.0  # Assumption\n    storage_capacity = [100, 150, 120, 110]  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n    Storage = [[LpVariable(f"Storage_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    storage_cost_term = lpSum(storage_cost * Storage[i][t] for i in products for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term - storage_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # New constraint: Storage capacity constraint\n    for i in products:\n        for t in periods:\n            problem += Storage[i][t] <= storage_capacity[i], f"Storage_Capacity_{i}_{t}"\n\n    # New strategy: Just-in-Time (JIT) production for product 0\n    for t in periods:\n        problem += Produced[0][t] == Sale[0][t], f"JIT_Production_{t}"\n\n    # New strategy: Safety stock for product 1\n    for t in periods:\n        problem += Stock[1][t] >= 10, f"Safety_Stock_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Storage": [[Storage[i][t].varValue for t in periods] for i in products],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Just-in-Time (JIT) production | Produce only what is needed to meet demand | Reduce inventory holding costs and minimize waste |\n# | Safety stock | Maintain a minimum inventory level for product 1 | Ensure availability of product 1 during periods of high demand |\n# | Storage capacity constraint | Limit storage capacity for each product | Prevent overstocking and reduce storage costs |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added three new strategies:\n\n1. Just-in-Time (JIT) production for product 0: This strategy aims to reduce inventory holding costs and minimize waste by producing only what is needed to meet demand.\n2. Safety stock for product 1: This strategy ensures availability of product 1 during periods of high demand by maintaining a minimum inventory level.\n3. Storage capacity constraint: This constraint limits storage capacity for each product to prevent overstocking and reduce storage costs.\n\nI also added a new decision variable `Storage` to represent the storage levels for each product and period. The objective function now includes a term for storage costs.\n\nNote that I assumed some additional data, such as storage cost and storage capacity, which may need to be adjusted based on the specific problem instance.',
 '\n\nHere is the modified code with additional strategies for storage, production, sales, and workforce:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # New strategy for storage: implement a minimum storage level\n    min_storage_level = 10.0\n    for i in products:\n        for t in periods:\n            problem += Stock[i][t] >= min_storage_level, f"Min_Storage_Level_{i}_{t}"\n\n    # New strategy for production: prioritize products with high profit margins\n    profit_margin = [p / (1 + h) for p, h in zip(profit, holding_costs)]\n    for t in periods:\n        problem += lpSum([Produced[i][t] * profit_margin[i] for i in products]) >= lpSum([Produced[i][t] for i in products]) * 0.8, f"Profit_Margin_Priority_{t}"\n\n    # New strategy for sales: limit sales to 90% of demand\n    for i in products:\n        for t in periods:\n            problem += Sale[i][t] <= 0.9 * max_demand[i][t], f"Sales_Limit_{i}_{t}"\n\n    # New strategy for workforce: hire/firing decisions based on workload\n    workload_threshold = 100.0\n    for t in periods:\n        problem += lpSum([Produced[i][t] * worker_hours_per_product for i in products]) <= Workforce[t] * workload_threshold, f"Workload_Balance_{t}"\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Min Storage Level | Ensure a minimum storage level to avoid stockouts | Addressing stockout issues in periods 2 and 3 |\n# | Profit Margin Priority | Prioritize products with high profit margins | Increasing overall profit by focusing on high-margin products |\n# | Sales Limit | Limit sales to 90% of demand to avoid over-selling | Preventing over-selling and subsequent stockouts |\n# | Workload-Based Hiring/Firing | Hire/fire workers based on workload to optimize labor costs | Reducing labor costs by adjusting workforce to match workload |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added four new strategies:\n\n1. **Min Storage Level**: Ensure a minimum storage level to avoid stockouts.\n2. **Profit Margin Priority**: Prioritize products with high profit margins to increase overall profit.\n3. **Sales Limit**: Limit sales to 90% of demand to prevent over-selling and subsequent stockouts.\n4. **Workload-Based Hiring/Firing**: Hire/fire workers based on workload to optimize labor costs.\n\nThese strategies address specific issues detected in the simulation feedback, such as stockouts, low profit margins, over-selling, and inefficient labor costs.',
 '\n\nHere is a revised version of the code that incorporates new strategies for storage, production, sales, and workforce. I\'ve added a new strategy for storage, which is to prioritize products with higher profit margins, and a new strategy for production, which is to produce more of products that have higher demand. I\'ve also added a new strategy for sales, which is to offer discounts for products that have high inventory levels. Additionally, I\'ve added a new strategy for workforce, which is to hire temporary workers during peak periods.\n\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # New strategy for storage: prioritize products with higher profit margins\n    priority = [profit[i] / holding_costs[i] for i in products]\n    for t in periods:\n        for i in products:\n            problem += Stock[i][t] * priority[i] <= Stock[i][t-1] * priority[i] + Produced[i][t] * priority[i], f"Priority_Stock_{i}_{t}"\n\n    # New strategy for production: produce more of products that have higher demand\n    for t in periods:\n        for i in products:\n            problem += Produced[i][t] >= max_demand[i][t] * 0.8, f"High_Demand_Production_{i}_{t}"\n\n    # New strategy for sales: offer discounts for products that have high inventory levels\n    for t in periods:\n        for i in products:\n            problem += Sale[i][t] >= Stock[i][t-1] * 0.2, f"Discount_Sales_{i}_{t}"\n\n    # New strategy for workforce: hire temporary workers during peak periods\n    for t in periods:\n        problem += Workforce[t] >= initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        problem += Hires[t] >= (Workforce[t] - initial_workers) * 0.5, f"Temporary_Workers_{t}"\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Priority Storage | Prioritize products with higher profit margins | To maximize profit by storing more valuable products |\n# | High Demand Production | Produce more of products that have higher demand | To meet customer demand and reduce lost sales |\n# | Discount Sales | Offer discounts for products that have high inventory levels | To clear inventory and reduce holding costs |\n# | Temporary Workers | Hire temporary workers during peak periods | To meet increased demand during peak periods |\n\n# Call the function to optimize production\noptimize_production()\n```\n\nThis code adds four new strategies:\n\n1. Priority Storage: Prioritize products with higher profit margins by allocating more storage space to them.\n2. High Demand Production: Produce more of products that have higher demand to meet customer demand and reduce lost sales.\n3. Discount Sales: Offer discounts for products that have high inventory levels to clear inventory and reduce holding costs.\n4. Temporary Workers: Hire temporary workers during peak periods to meet increased demand during peak periods.\n\nThese strategies are implemented by adding new constraints and objective function terms to the LP model. The code also includes a table to describe the new strategies and their reasons for implementation.',
 '\n\nHere is a revised version of the code with additional strategies:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n    Storage = [[LpVariable(f"Storage_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    storage_cost_term = lpSum(5 * Storage[i][t] for i in products for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term - storage_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            # Storage constraint\n            problem += Storage[i][t] <= 0.5 * Stock[i][t], f"Storage_Constraint_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # New strategy for storage: limit storage to 50% of stock\n    for i in products:\n        for t in periods:\n            problem += Storage[i][t] <= 0.5 * Stock[i][t], f"Storage_Limit_{i}_{t}"\n\n    # New strategy for production: prioritize products with high profit margin\n    for t in periods:\n        problem += lpSum(profit[i] * Produced[i][t] for i in products) >= 0.7 * lpSum(profit[i] * max_demand[i][t] for i in products), f"Profit_Margin_{t}"\n\n    # New strategy for sales: incentivize sales of products with low inventory\n    for i in products:\n        for t in periods:\n            problem += Sale[i][t] >= 0.8 * initial_inventory[i] if t == 0 else 0.8 * Stock[i][t-1], f"Sales_Incentive_{i}_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Storage": [[Storage[i][t].varValue for t in periods] for i in products],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Storage Limit | Limit storage to 50% of stock | To avoid overstocking and reduce holding costs |\n# | Prioritize High-Profit Products | Prioritize production of products with high profit margin | To increase overall profit |\n# | Sales Incentive | Incentivize sales of products with low inventory | To reduce inventory levels and avoid stockouts |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added a new strategy for sales, which incentivizes the sale of products with low inventory levels. This strategy is implemented by adding a constraint that ensures sales of products with low inventory levels are at least 80% of the initial inventory or previous period\'s stock level.',
 '\n\nHere is the modified Python code that adds new strategies for storage, production, and sales:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # New strategy for storage: implement a minimum storage level for each product\n    min_storage_level = 10.0\n    for i in products:\n        for t in periods:\n            problem += Stock[i][t] >= min_storage_level, f"Min_Storage_{i}_{t}"\n\n    # New strategy for production: prioritize production of high-demand products\n    high_demand_products = [1, 3]\n    for t in periods:\n        problem += lpSum(Produced[i][t] for i in high_demand_products) >= lpSum(Produced[i][t] for i in products if i not in high_demand_products), f"Prioritize_High_Demand_{t}"\n\n    # New strategy for sales: implement a maximum sales limit for each product\n    max_sales_limit = 200.0\n    for i in products:\n        for t in periods:\n            problem += Sale[i][t] <= max_sales_limit, f"Max_Sales_{i}_{t}"\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Min storage level | Implement a minimum storage level for each product | To avoid stockouts and ensure a minimum level of inventory |\n# | Prioritize high-demand products | Prioritize production of high-demand products | To maximize revenue and meet demand |\n# | Max sales limit | Implement a maximum sales limit for each product | To avoid overstocking and reduce holding costs |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added three new strategies:\n\n1. **Min storage level**: Implement a minimum storage level for each product to avoid stockouts and ensure a minimum level of inventory.\n2. **Prioritize high-demand products**: Prioritize production of high-demand products to maximize revenue and meet demand.\n3. **Max sales limit**: Implement a maximum sales limit for each product to avoid overstocking and reduce holding costs.\n\nThese strategies are implemented using additional constraints and objective function terms. The new strategies are described in the table at the end of the code.',
 '```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <=                        capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Call the function to optimize production\noptimize_production()\n```',
 '\n\nHere is the modified code that incorporates new strategies for storage, production, sales, and workforce:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # New strategy for storage: Implement a safety stock policy for products 0 and 2\n    safety_stock = 10\n    for i in [0, 2]:\n        for t in periods:\n            problem += Stock[i][t] >= safety_stock, f"Safety_Stock_{i}_{t}"\n\n    # New strategy for production: Implement a batch production policy for product 1\n    batch_size = 50\n    for t in periods:\n        problem += Produced[1][t] >= batch_size, f"Batch_Production_{1}_{t}"\n\n    # New strategy for sales: Implement a price discount policy for product 3\n    discount_rate = 0.1\n    for t in periods:\n        problem += Sale[3][t] >= min_sales[3][t] * (1 - discount_rate), f"Price_Discount_{3}_{t}"\n\n    # New strategy for workforce: Implement a flexible workforce policy\n    flexible_workforce = 5\n    for t in periods:\n        problem += Workforce[t] >= flexible_workforce, f"Flexible_Workforce_{t}"\n\n    # New strategy for production: Implement a just-in-time production policy for product 2 in period 3\n    just_in_time_production = 100\n    problem += Produced[2][3] == just_in_time_production, f"Just_In_Time_Production_{2}_{3}"\n\n    # New strategy for sales: Implement a promotional sales policy for product 1 in period 2\n    promotional_sales = 150\n    problem += Sale[1][2] == promotional_sales, f"Promotional_Sales_{1}_{2}"\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Safety Stock | Implement a safety stock policy for products 0 and 2 | To mitigate stockouts and ensure a minimum level of inventory |\n# | Batch Production | Implement a batch production policy for product 1 | To reduce production costs and improve efficiency |\n# | Price Discount | Implement a price discount policy for product 3 | To increase sales and revenue |\n# | Flexible Workforce | Implement a flexible workforce policy | To improve workforce flexibility and reduce labor costs |\n# | Just-In-Time Production | Implement a just-in-time production policy for product 2 in period 3 | To reduce inventory holding costs and improve supply chain efficiency |\n# | Promotional Sales | Implement a promotional sales policy for product 1 in period 2 | To increase sales and revenue |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added two new strategies:\n\n1. **Just-In-Time Production**: Implement a just-in',
 '```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <=                        capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Call the function to optimize production\noptimize_production()\n```']

In [ ]:
import re
import ast
import sys
import io
import random

# Function to extract matrices from the solution output
def extract_matrices(execp):
    patterns = {
        "Produced": r"Produced: (\[\[.*?\]\])",
        "Sale": r"Sale: (\[\[.*?\]\])",
        "Stock": r"Stock: (\[\[.*?\]\])",
        "Overtime": r"Overtime: (\[.*?\])",
        "Workforce": r"Workforce: (\[.*?\])",
        "Hires": r"Hires: (\[.*?\])",
        "Firings": r"Firings: (\[.*?\])",
        "Profit": r"Profit: (.*?)\n"
    }

    matrices = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, execp, re.DOTALL)
        if match:
            matrices[key] = ast.literal_eval(match.group(1))

    return matrices

# Initialize lists to store feedback and results
feedback_list = []
result_list = []

# Function to extract Python code manually

# Loop through each solution in `collect_array`
for solution in collect_array:
    # Extract Python code from the solution
    py_code_p = extract_python_code_manual(solution)

    # Redirect stdout to capture prints
    old_stdout = sys.stdout  # Save the old state
    new_stdout = io.StringIO()
    sys.stdout = new_stdout

    # Execute the extracted Python code
    exec(py_code_p[0])

    # Restore stdout to its original state
    sys.stdout = old_stdout
    execp = new_stdout.getvalue()

    # Extract matrices
    matrices = extract_matrices(execp)

    # Run the simulation
    produced = matrices.get("Produced")
    sale = matrices.get("Sale")
    stock = matrices.get("Stock")
    overtime = matrices.get("Overtime")
    workers = matrices.get("Workforce")
    hired = matrices.get("Hires")
    fired = matrices.get("Firings")

    seed = 42

    previous_feedback = run_extended_simulation(produced, sale, overtime, workers, hired, fired, initial_inventory, seed=seed)[:-1]
    previous_res = run_extended_simulation(produced, sale, overtime, workers, hired, fired, initial_inventory, seed=seed)[-1]

    # Store the feedback and result
    feedback_list.append(previous_feedback)
    result_list.append(previous_res)

# Now `feedback_list` contains feedback for each solution
# and `result_list` contains the results for each solution


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [ ]:
result_list

NameError: name 'result_list' is not defined

In [ ]:
import re

# Provided profits list
profits_list = [
    '\nTotal Profits across all products and periods: 476751.98',
    '\nTotal Profits across all products and periods: 478961.86',
    '\nTotal Profits across all products and periods: 448179.08',
    '\nTotal Profits across all products and periods: 606882.68',
    '\nTotal Profits across all products and periods: 711626.13',
    '\nTotal Profits across all products and periods: 507977.15',
    '\nTotal Profits across all products and periods: 442044.71',
    '\nTotal Profits across all products and periods: 587904.90',
    '\nTotal Profits across all products and periods: 442044.71'
]

# Function to extract profit from the string
def extract_profit(profit_str):
    match = re.search(r'Total Profits across all products and periods: ([\d\.]+)', profit_str)
    if match:
        return float(match.group(1))
    return 0

# Extract profits and pair with original indices
profits_with_indices = [(i, extract_profit(profit_str)) for i, profit_str in enumerate(profits_list)]

# Sort by profit in descending order
sorted_profits = sorted(profits_with_indices, key=lambda x: x[1], reverse=True)

# Select the top 4 solutions
top_4_indices = [index for index, profit in sorted_profits[:4]]

# Keep only the top 4 entries in collect_array
collect_array_top_4 = [collect_array[i] for i in top_4_indices]

feedback_list_top_4 = [feedback_list[i] for i in top_4_indices]

# Now collect_array_top_4 contains only the top 4 performing solutions


In [ ]:
feedback_list_top_4

[([{'Period': 0,
    'Products': [{'Product ID': 0,
      'Produced': 30.0,
      'Sold': 71.0,
      'Missed Sales': 1.9798605381019456,
      'Ending Inventory': 0.0,
      'Period Profit': 8449.0},
     {'Product ID': 1,
      'Produced': 23.333333333333332,
      'Sold': 76.29117815185818,
      'Missed Sales': 0.0,
      'Ending Inventory': 5.042155181475152,
      'Period Profit': 60560.7278374607},
     {'Product ID': 2,
      'Produced': 0.0,
      'Sold': 23.684145419110834,
      'Missed Sales': 0.0,
      'Ending Inventory': 7.315854580889166,
      'Period Profit': 4863.932719299938},
     {'Product ID': 3,
      'Produced': 0.0,
      'Sold': 28.71712128794484,
      'Missed Sales': 0.0,
      'Ending Inventory': 9.282878712055162,
      'Period Profit': 3267.9428887207737}],
    'Overtime Hours': 106.66666666666666,
    'Regular Hours': 0.0,
    'Hired': 0.0,
    'Fired': 0,
    'Workforce': 0.0},
   {'Period': 1,
    'Products': [{'Product ID': 0,
      'Produced': 151.0

In [ ]:
collect_array_top_4

['\n\nHere is a revised version of the code with additional strategies:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n       

In [ ]:
import numpy as np
import math
import random
import re
import os
import sys
import io
import ast
import replicate

def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb
    return winner_profit, winner_input, winner_fb

def chance(ps, pf, explain_example, max_executions=10, improvement_threshold=0.01, window_size=3):
    previous_solution = ps
    previous_feedback = pf
    counter = 0
    seed = 42
    max_retry_attempts = 3

    recent_profits = []
    best_profit = float('-inf')

    while counter < max_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                exec(py_code_s[0])

                sys.stdout = old_stdout

                break
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        execs = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        if winner_profit > best_profit:
            best_profit = winner_profit

        recent_profits.append(winner_profit)
        if len(recent_profits) > window_size:
            recent_profits.pop(0)

        if len(recent_profits) == window_size:
            improvement = (recent_profits[-1] - recent_profits[0]) / recent_profits[0]
            if improvement < improvement_threshold:
                print(f"No significant improvement detected over the last {window_size} iterations. Stopping early.")
                break

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return winner_input


In [ ]:
chat_history = [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": f"""Here you see two produciton planning models, incl. simulaiton feedback. Model 1: {model1} and feedback model 1: {feedback1} and here is model 2: {model2} and feedback of model 2: {feedback2}. Now find a merged optimization model that may be better than the previous 2 models combining their strenghts, while also taking simulation feedback into account"""}
]

response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
print(response.choices[0].message.content)

# Save the alternative solution in the secondary_solutions list
previous_solution = response.choices[0].message.content

In [ ]:
# New try, also include non LP
#def chance(ps, pf, explain_example, number_of_executions = 2):
#    previous_solution = ps
#    previous_feedback = pf

feedback_list_top_4
collect_array_top_4

i = 0
arr = []

while i < len(collect_array_top_4):

    model1 = collect_array_top_4[i]
    feedback1 = feedback_list_top_4[i]
    model2 = collect_array_top_4[i+1]
    feedback2 = feedback_list_top_4[i+1]

    counter = 0
    seed = 42

    #input = {
    #"prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, in the end, don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX, Stock: XXX. If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
    #"max_tokens": 2000  # Set the max_tokens parameter for output
    #}
    # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
    # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",


    input = {
    "prompt": f"""Here you see two produciton planning models, incl. simulaiton feedback. Model 1: {model1} and feedback model 1: {feedback1} and here is model 2: {model2} and feedback of model 2: {feedback2}. Now find a merged optimization model that may be better than the previous 2 models combining their strenghts, while also taking simulation feedback into account""",
    # "max_tokens": 2500  # Set the max_tokens parameter for output max_new_tokens
    "max_new_tokens": 2500
    }


    # Make the prediction using the correct method to pass the token
    client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

    # Make the prediction
    prediction = client.run(
        "meta/meta-llama-3-70b-instruct",
        input=input
    )

    entry = prediction
    entry = "".join(entry)

    arr.append(entry)

    print(entry)

    i = i+2

NameError: name 'replicate' is not defined

In [ ]:

def extract_python_code_manual(text):
    """
    Manually extracts Python code enclosed between ```python and ``` markers.

    Parameters:
    - text (str): The input text containing the Python code blocks.

    Returns:
    - list of str: A list containing all extracted Python code blocks.
    """
    code_blocks = []
    capture = False
    current_block = []

    # Split the text into lines for line-by-line processing
    lines = text.split('\n')

    for line in lines:
        # Check if the line contains the start marker
        if line.strip() == '```python':
            capture = True
            current_block = []  # Start a new block
        elif line.strip() == '```' and capture:
            # If we find the end marker and are in capture mode, stop capturing
            capture = False
            # Add the current block to the list of code blocks
            code_blocks.append('\n'.join(current_block))
        elif capture:
            # If we are in capture mode, add the line to the current block
            current_block.append(line)

    return code_blocks

In [ ]:
# New try, also include non LP
#def chance(ps, pf, explain_example, number_of_executions = 2):
#    previous_solution = ps
#    previous_feedback = pf

feedback_list_top_4
collect_array_top_4

i = 0
arr = []

while i < len(collect_array_top_4):

    model1 = collect_array_top_4[i]
    feedback1 = feedback_list_top_4[i]
    model2 = collect_array_top_4[i+1]
    feedback2 = feedback_list_top_4[i+1]

    counter = 0
    seed = 42

    #input = {
    #"prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, in the end, don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX, Stock: XXX. If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
    #"max_tokens": 2000  # Set the max_tokens parameter for output
    #}
    # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
    # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",


    chat_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""Here you see two produciton planning models, incl. simulaiton feedback. Model 1: {model1} and feedback model 1: {feedback1} and here is model 2: {model2} and feedback of model 2: {feedback2}. Now find a merged optimization model that may be better than the previous 2 models combining their strenghts, while also taking simulation feedback into account. The code must be denoted as ```python in the beginning of the code and ``` at the end of the code!!! Also don't forget to include the Table: New Strategies with all the strategies included. Create an executalbe production planning model. Buy the way: Monte Carlo Tree Search says that the optimal produciton plan is {optimal_plan}. You can use this as a northern star to steer the optimiziton model into that direction. Make sure that the code is executable"""}
    ]

    response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
    print(response.choices[0].message.content)

    # Save the alternative solution in the secondary_solutions list
    entry = response.choices[0].message.content

    arr.append(entry)

    i = i+2

```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus
import random

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],
        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],
 

In [ ]:
extract_python_code_manual(arr[1])

['from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.

In [ ]:
x = extract_python_code_manual(arr[0])

PulpError: overlapping constraint names: Workforce_Balance_0

In [ ]:
exec(x[0])

PulpError: overlapping constraint names: Workforce_Balance_0

In [ ]:

# Function to execute Python code
def execute_python_code(code):
    exec(code[0])

i = 0
arr = []

while i < len(collect_array_top_4):
    model1 = collect_array_top_4[i]
    feedback1 = feedback_list_top_4[i]
    model2 = collect_array_top_4[i + 1]
    feedback2 = feedback_list_top_4[i + 1]

    success = False
    while not success:
        chat_history = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"""Here you see two production planning models, incl. simulation feedback. Model 1: {model1} and feedback model 1: {feedback1} and here is model 2: {model2} and feedback of model 2: {feedback2}. Now find a merged optimization model that may be better than the previous 2 models combining their strengths, while also taking simulation feedback into account. The code must be denoted as ```python in the beginning of the code and ``` at the end of the code!!! Also don't forget to include the Table: New Strategies with all the strategies included. Create an executable production planning model. By the way: Monte Carlo Tree Search says that the optimal production plan is {optimal_plan}. You can use this as a northern star to steer the optimization model into that direction. Make sure that the code is executable."""}
        ]

        response = openai.ChatCompletion.create(model="gpt-4o", messages=chat_history)
        response_content = response.choices[0].message.content

        try:
            code = extract_python_code_manual(response_content)
            execute_python_code(code)  # Ensure the code is executable
            arr.append(response_content)
            success = True
        except Exception as e:
            print(f"Error executing generated code: {e}")

    i += 2


Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: local variable 'value' referenced before assignment
Error executing generated code: 'LpVariable' object is not subscriptable
Error executing generated code: local variable 'value' referenced before assignment
Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: overlapping constraint names: Workforce_Balance_0
Error executing generated code: local variable 'value' referenced before assignment
Error executing generated code: local variable 'value' referenced before assignment
Error executing g

In [ ]:
arr

['```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum, pulp\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2

Last mutations

In [ ]:
two_winner_versions = ['```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum, pulp\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Merged_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n    Storage = [[LpVariable(f"Storage_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    storage_cost_term = lpSum(5 * Storage[i][t] for i in products for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term - storage_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            # Storage constraint\n            problem += Storage[i][t] <= 0.5 * Stock[i][t], f"Storage_Constraint_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # New strategy for storage: limit storage to 50% of stock\n    for i in products:\n        for t in periods:\n            problem += Storage[i][t] <= 0.5 * Stock[i][t], f"Storage_Limit_{i}_{t}"\n\n    # New strategy for production: prioritize products with high profit margin\n    for t in periods:\n        problem += lpSum(profit[i] * Produced[i][t] for i in products) >= 0.7 * lpSum(profit[i] * max_demand[i][t] for i in products), f"Profit_Margin_Production_{t}"\n\n    # New strategy for sales: incentivize sales of products with low inventory\n    for i in products:\n        for t in periods:\n            problem += Sale[i][t] >= 0.8 * initial_inventory[i] if t == 0 else 0.8 * Stock[i][t-1], f"Sales_Incentive_{i}_{t}"\n\n    # New strategy for production: produce more of products that have higher demand\n    for t in periods:\n        for i in products:\n            problem += Produced[i][t] >= max_demand[i][t] * 0.8, f"High_Demand_Production_{i}_{t}"\n\n    # New strategy for sales: offer discounts for products that have high inventory levels\n    for t in periods:\n        for i in products:\n            problem += Sale[i][t] >= Stock[i][t-1] * 0.2, f"Discount_Sales_{i}_{t}"\n\n    # New strategy for workforce: hire temporary workers during peak periods\n    for t in periods:\n        problem += Hires[t] >= (Workforce[t] - initial_workers) * 0.5, f"Temporary_Workers_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Storage": [[Storage[i][t].varValue for t in periods] for i in products],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Storage Limit | Limit storage to 50% of stock | To avoid overstocking and reduce holding costs |\n# | Prioritize High-Profit Products | Prioritize production of products with high profit margin | To increase overall profit |\n# | Sales Incentive | Incentivize sales of products with low inventory | To reduce inventory levels and avoid stockouts |\n# | High Demand Production | Produce more of products that have higher demand | To meet customer demand and reduce lost sales |\n# | Discount Sales | Offer discounts for products that have high inventory levels | To clear inventory and reduce holding costs |\n# | Temporary Workers | Hire temporary workers during peak periods | To meet increased demand during peak periods |\n\n# Call the function to optimize production\noptimize_production()\n```',
 '```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Strategy: Implement a safety stock policy for products 0 and 2\n    safety_stock = 10\n    for i in [0, 2]:\n        for t in periods:\n            problem += Stock[i][t] >= safety_stock, f"Safety_Stock_{i}_{t}"\n\n    # Strategy: Implement a batch production policy for product 1\n    batch_size = 50\n    for t in periods:\n        problem += Produced[1][t] >= batch_size, f"Batch_Production_{1}_{t}"\n\n    # Strategy: Implement a price discount policy for product 3\n    discount_rate = 0.1\n    for t in periods:\n        problem += Sale[3][t] >= min_sales[3][t] * (1 - discount_rate), f"Price_Discount_{3}_{t}"\n\n    # Strategy: Implement a flexible workforce policy\n    flexible_workforce = 5\n    for t in periods:\n        problem += Workforce[t] >= flexible_workforce, f"Flexible_Workforce_{t}"\n\n    # Strategy: Implement a just-in-time production policy for product 2 in period 3\n    just_in_time_production = 100\n    problem += Produced[2][3] == just_in_time_production, f"Just_In_Time_Production_{2}_{3}"\n\n    # Strategy: Implement a promotional sales policy for product 1 in period 2\n    promotional_sales = 150\n    problem += Sale[1][2] == promotional_sales, f"Promotional_Sales_{1}_{2}"\n\n    # Strategy: Implement a minimum storage level for each product\n    min_storage_level = 10.0\n    for i in products:\n        for t in periods:\n            problem += Stock[i][t] >= min_storage_level, f"Min_Storage_{i}_{t}"\n\n    # Strategy: Prioritize production of high-demand products\n    high_demand_products = [1, 3]\n    for t in periods:\n        problem += lpSum(Produced[i][t] for i in high_demand_products) >= lpSum(Produced[i][t] for i in products if i not in high_demand_products), f"Prioritize_High_Demand_{t}"\n\n    # Strategy: Implement a maximum sales limit for each product\n    max_sales_limit = 200.0\n    for i in products:\n        for t in periods:\n            problem += Sale[i][t] <= max_sales_limit, f"Max_Sales_{i}_{t}"\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Safety Stock | Implement a safety stock policy for products 0 and 2 | To mitigate stockouts and ensure a minimum level of inventory |\n# | Batch Production | Implement a batch production policy for product 1 | To reduce production costs and improve efficiency |\n# | Price Discount | Implement a price discount policy for product 3 | To increase sales and revenue |\n# | Flexible Workforce | Implement a flexible workforce policy | To improve workforce flexibility and reduce labor costs |\n# | Just-In-Time Production | Implement a just-in-time production policy for product 2 in period 3 | To reduce inventory holding costs and improve supply chain efficiency |\n# | Promotional Sales | Implement a promotional sales policy for product 1 in period 2 | To increase sales and revenue |\n# | Min storage level | Implement a minimum storage level for each product | To avoid stockouts and ensure a minimum level of inventory |\n# | Prioritize high-demand products | Prioritize production of high-demand products | To maximize revenue and meet demand |\n# | Max sales limit | Implement a maximum sales limit for each product | To avoid overstocking and reduce holding costs |\n\n# Call the function to optimize production\noptimize_production()\n```']

In [ ]:
import numpy as np
import math
import random
import re
import os
import sys
import io
import ast
import replicate

def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb
    return winner_profit, winner_input, winner_fb

def chance(ps, pf, explain_example, max_executions=10, improvement_threshold=0.01, window_size=3):
    previous_solution = ps
    previous_feedback = pf
    counter = 0
    seed = 42
    max_retry_attempts = 3

    recent_profits = []
    best_profit = float('-inf')

    while counter < max_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                exec(py_code_s[0])

                sys.stdout = old_stdout

                break
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        execs = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        if winner_profit > best_profit:
            best_profit = winner_profit

        recent_profits.append(winner_profit)
        if len(recent_profits) > window_size:
            recent_profits.pop(0)

        if len(recent_profits) == window_size:
            improvement = (recent_profits[-1] - recent_profits[0]) / recent_profits[0]
            if improvement < improvement_threshold:
                print(f"No significant improvement detected over the last {window_size} iterations. Stopping early.")
                break

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return winner_input


In [ ]:
import re
import ast
import sys
import io
import random

# Function to extract matrices from the solution output
def extract_matrices(execp):
    patterns = {
        "Produced": r"Produced: (\[\[.*?\]\])",
        "Sale": r"Sale: (\[\[.*?\]\])",
        "Stock": r"Stock: (\[\[.*?\]\])",
        "Overtime": r"Overtime: (\[.*?\])",
        "Workforce": r"Workforce: (\[.*?\])",
        "Hires": r"Hires: (\[.*?\])",
        "Firings": r"Firings: (\[.*?\])",
        "Profit": r"Profit: (.*?)\n"
    }

    matrices = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, execp, re.DOTALL)
        if match:
            matrices[key] = ast.literal_eval(match.group(1))

    return matrices

# Initialize lists to store feedback and results
feedback_list = []
result_list = []

# Extract Python code from the solution
py_code_p = extract_python_code_manual(two_winner_versions[1])

# Redirect stdout to capture prints
old_stdout = sys.stdout  # Save the old state
new_stdout = io.StringIO()
sys.stdout = new_stdout

# Execute the extracted Python code
exec(py_code_p[0])

# Restore stdout to its original state
sys.stdout = old_stdout
execp = new_stdout.getvalue()

# Extract matrices
matrices = extract_matrices(execp)

# Run the simulation
produced = matrices.get("Produced")
sale = matrices.get("Sale")
stock = matrices.get("Stock")
overtime = matrices.get("Overtime")
workers = matrices.get("Workforce")
hired = matrices.get("Hires")
fired = matrices.get("Firings")

seed = 42

previous_feedback = run_extended_simulation(produced, sale, overtime, workers, hired, fired, initial_inventory, seed=seed)[:-1]
previous_res = run_extended_simulation(produced, sale, overtime, workers, hired, fired, initial_inventory, seed=seed)[-1]

# Store the feedback and result
feedback_list.append(previous_feedback)
result_list.append(previous_res)

# Now `feedback_list` contains feedback for each solution
# and `result_list` contains the results for each solution


In [ ]:
feedback_list[0]

([{'Period': 0,
   'Products': [{'Product ID': 0,
     'Produced': 0.0,
     'Sold': 41.0,
     'Missed Sales': 1.1432997473546465,
     'Ending Inventory': 0.0,
     'Period Profit': 4879.0},
    {'Product ID': 1,
     'Produced': 33.333333333333336,
     'Sold': 91.33333333333334,
     'Missed Sales': 91.4771011776624,
     'Ending Inventory': 0.0,
     'Period Profit': 72610.00000000001},
    {'Product ID': 2,
     'Produced': 0.0,
     'Sold': 23.684145419110834,
     'Missed Sales': 0.0,
     'Ending Inventory': 7.315854580889166,
     'Period Profit': 4863.932719299938},
    {'Product ID': 3,
     'Produced': 0.0,
     'Sold': 28.71712128794484,
     'Missed Sales': 0.0,
     'Ending Inventory': 9.282878712055162,
     'Period Profit': 3267.9428887207737}],
   'Overtime Hours': 56.66666666666667,
   'Regular Hours': 10.0,
   'Hired': 0.0,
   'Fired': 0,
   'Workforce': 10.0},
  {'Period': 1,
   'Products': [{'Product ID': 0,
     'Produced': 182.4,
     'Sold': 40.00664007621306,

In [ ]:
# Define a function to run parallel tasks, before, the setting was 5/3   max_executions=10
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=5, max_executions=10):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, max_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

In [ ]:
x = run_parallel_tasks(two_winner_versions[0], feedback_list[0], explain_example, number_of_iterations=1, max_executions=5)

Execution 1
Requesting prediction...


/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Prediction received.
Secondary solution received.
Executing previous solution...
Previous solution executed.
Executing secondary solution...


In [ ]:
feedb2 = str(feedback_list[0])

In [ ]:
es1 = chance(two_winner_versions[0], feedb, explain_example, number_of_executions=5)
es2 = chance(two_winner_versions[1], feedb2, explain_example, number_of_executions=5)

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [ ]:
es1

'```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum, pulp\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 20

In [ ]:
es1

NameError: name 'es1' is not defined

PROMP for crossover

In [ ]:
"""Here you see two produciton planning models, incl. simulaiton feedback: {model 1} and feedback [{model 1} and here is model 2: {model 2} and feedback 2: {feedback2}. Now find a merged solution that may be better than the previous 2 models combining their strenghts, while also taking simulation feedback into account"""

In [ ]:
result_list

['\nTotal Profits across all products and periods: 571978.19']

In [ ]:
optimal_plan = """
([[374.54011884736246,
   950.7143064099162,
   731.9939418114051,
   598.6584841970366,
   156.01864044243652,
   155.99452033620264],
  [58.08361216819946,
   866.1761457749352,
   601.1150117432088,
   708.0725777960455,
   20.584494295802447,
   969.9098521619943],
  [832.4426408004217,
   212.33911067827614,
   181.82496720710063,
   183.4045098534338,
   304.24224295953775,
   524.7564316322379],
  [431.94501864211577,
   291.22914019804193,
   611.8528947223795,
   139.49386065204183,
   292.14464853521815,
   366.3618432936917]],
 [[456.06998421703594,
   785.1759613930136,
   199.67378215835973,
   514.2344384136117,
   592.4145688620424,
   46.45041271999772],
  [607.5448519014384,
   170.52412368729154,
   65.05159298527951,
   948.8855372533333,
   965.6320330745593,
   808.3973481164611],
  [304.6137691733707,
   97.67211400638386,
   684.2330265121569,
   440.1524937396013,
   122.03823484477883,
   495.17691011127016],
  [34.388521115218396,
   909.3204020787821,
   258.7799816000169,
   662.522284353982,
   311.71107608941094,
   520.0680211778108]],
 [546.7102793432797,
  184.85445552552704,
  969.5846277645586,
  775.1328233611146,
  939.4989415641891,
  894.8273504276489],
 [1195.7999576221703,
  1843.7484700462337,
  176.985004103839,
  391.9657248382904,
  90.45457782107613,
  650.6606615265287],
 [777.354579378964,
  542.6980635477918,
  1657.4750183038586,
  713.5066533871785,
  561.8690193747615,
  1085.392166316497],
 [281.8484499495253,
  1604.3939615080794,
  149.10128735954166,
  1973.7738732010346,
  1544.4895385933148,
  397.4313630683448])

  for the input factors: produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev
  """

TEXT

BBB

In [ ]:
collect_array = []

number_of_iterations = 20
count = 0

while count < number_of_iterations:
    collect_array.append(chance(previous_solution_bu, previous_feedback_bu, explain_example, number_of_executions=5))
    count += 1

print(collect_array)



Here is the modified code with new strategies for storage, production, sales, and workforce:
```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 

In [ ]:
# Define a function to run parallel tasks, before, the setting was 5/3   max_executions=10
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=10, max_executions=5):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, max_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

In [ ]:
collect_array

['\n\nHere is the modified code with a new strategy for storage:\n```python\nimport pulp\n\ndef supply_chain_optimization_extended(\n    time_horizon, products_total, workstations_total, profit, holding_costs, min_sales, max_demand,\n    production_time, capacity, capacity_reduction_factor=0.9, initial_inventory=None, yield_loss=None, \n    worker_hours_per_product=None, hourly_wage=None, hourly_wage_overtime=None, hiring_cost=None, \n    layoff_cost=None, initial_workers=None, resource_requirement=None, resource_capacity=None):\n\n    # Create the problem\n    problem = pulp.LpProblem("Supply_Chain_Optimization_Extended", pulp.LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Amount_produced = [[pulp.LpVariable(f"Amount_produced_product_{i}_period_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in 

In [ ]:
import numpy as np
import math
import random
import re
import os
import sys
import io
import ast
import replicate

def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb
    return winner_profit, winner_input, winner_fb

def chance(ps, pf, explain_example, max_executions=10, improvement_threshold=0.01, window_size=3):
    previous_solution = ps
    previous_feedback = pf
    counter = 0
    seed = 42

    recent_profits = []
    best_profit = float('-inf')

    while counter < max_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while True:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )
                cache_prediction = prediction
                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout
                exec(py_code_s[0])
                sys.stdout = old_stdout

                attempt += 1
                break
            except:
                print(f'Failed attempt {attempt}!')

        py_code_p = extract_python_code_manual(previous_solution)

        execs = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        if winner_profit > best_profit:
            best_profit = winner_profit

        recent_profits.append(winner_profit)
        if len(recent_profits) > window_size:
            recent_profits.pop(0)

        if len(recent_profits) == window_size:
            improvement = (recent_profits[-1] - recent_profits[0]) / recent_profits[0]
            if improvement < improvement_threshold:
                print(f"No significant improvement detected over the last {window_size} iterations. Stopping early.")
                break

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return winner_input


In [ ]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
import concurrent.futures
import replicate
import time
import sys
import io
import re
import ast

In [ ]:
#Extended additional INLSOP
def chance(ps, pf, explain_example, number_of_executions=5):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42
    max_retry_attempts = 3

    while counter < number_of_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new long-term/highlevel and logically interpretable storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
            # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Also use Mixed-Integer Non-Linear Programming (MINLP) for objective function and constraints using scipy package which I installed already (numpy and pulp installed already too). You can also try out non-liner constraints!. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2800
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                # Make the prediction
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                # Extracting the python code
                py_code_s = extract_python_code_manual(secondary_solution)

                # Redirect stdout to capture prints
                old_stdout = sys.stdout  # Save the old state
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                # Execute the code
                exec(py_code_s[0])

                # Restore stdout to its original state
                sys.stdout = old_stdout

                break  # Exit the retry loop if successful
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        # If the maximum number of retry attempts is reached, return the current winner version
        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        # Get the output and work with it
        execs = new_stdout.getvalue()

        # Redirect stdout to capture prints
        old_stdout = sys.stdout  # Save the old state
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        # Execute the code
        exec(py_code_p[0])
        # Restore stdout to its original state
        sys.stdout = old_stdout
        # Get the output and work with it
        execp = new_stdout.getvalue()

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")

        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")


        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        # Example operation
        print(f"Execution {counter + 1}")

        # Increment the counter at the end of each loop iteration
        counter += 1
        seed += 1

    return winner_input


In [ ]:
def chance(ps, pf, explain_example, number_of_executions=2):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42
    max_retry_attempts = 3

    while counter < number_of_executions:
        input = {
            # "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current LP model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                # Make the prediction
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                # Extracting the python code
                py_code_s = extract_python_code_manual(secondary_solution)

                # Redirect stdout to capture prints
                old_stdout = sys.stdout  # Save the old state
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                # Execute the code
                exec(py_code_s[0])

                # Restore stdout to its original state
                sys.stdout = old_stdout

                break  # Exit the retry loop if successful
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        # If the maximum number of retry attempts is reached, return the current winner version
        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        # Get the output and work with it
        execs = new_stdout.getvalue()

        # Redirect stdout to capture prints
        old_stdout = sys.stdout  # Save the old state
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        # Execute the code
        exec(py_code_p[0])
        # Restore stdout to its original state
        sys.stdout = old_stdout
        # Get the output and work with it
        execp = new_stdout.getvalue()

        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")

        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")


        # Define regex patterns for each matrix
        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        # Dictionary to hold the extracted matrices
        matrices = {}

        # Extract and evaluate each matrix
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        # Access the matrices
        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        # Example operation
        print(f"Execution {counter + 1}")

        # Increment the counter at the end of each loop iteration
        counter += 1
        seed += 1

    return winner_input


In [ ]:
import numpy as np
import math
import random
import re
import os
import sys
import io
import ast
import replicate

def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb
    return winner_profit, winner_input, winner_fb

def chance(ps, pf, explain_example, max_executions=10, improvement_threshold=0.01, window_size=3):
    previous_solution = ps
    previous_feedback = pf
    counter = 0
    seed = 42
    max_retry_attempts = 3

    recent_profits = []
    best_profit = float('-inf')

    while counter < max_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                exec(py_code_s[0])

                sys.stdout = old_stdout

                break
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        execs = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        if winner_profit > best_profit:
            best_profit = winner_profit

        recent_profits.append(winner_profit)
        if len(recent_profits) > window_size:
            recent_profits.pop(0)

        if len(recent_profits) == window_size:
            improvement = (recent_profits[-1] - recent_profits[0]) / recent_profits[0]
            if improvement < improvement_threshold:
                print(f"No significant improvement detected over the last {window_size} iterations. Stopping early.")
                break

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return winner_input


In [ ]:
import numpy as np
import math
import random
import re
import os
import sys
import io
import ast
import replicate
from scipy import stats

def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb
    return winner_profit, winner_input, winner_fb

def perform_t_test(profits):
    if len(profits) < 2:
        return False, 1.0
    t_stat, p_value = stats.ttest_rel(profits[:-1], profits[1:])
    return t_stat, p_value

def chance_with_t_test(ps, pf, explain_example, max_executions=10, improvement_threshold=0.01, window_size=3):
    previous_solution = ps
    previous_feedback = pf
    counter = 0
    seed = 42
    max_retry_attempts = 3

    recent_profits = []
    best_profit = float('-inf')

    while counter < max_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                exec(py_code_s[0])

                sys.stdout = old_stdout

                break
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        execs = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        if winner_profit > best_profit:
            best_profit = winner_profit

        recent_profits.append(winner_profit)
        if len(recent_profits) > window_size:
            recent_profits.pop(0)

        # Perform t-test
        t_stat, p_value = perform_t_test(recent_profits)
        if p_value > 0.05:
            print("No statistically significant improvement. Stopping early.")
            break

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return winner_input

# Sample usage
# ps = "Initial solution string"
# pf = "Initial feedback string"
# explain_example = "Explain example string"
# result = chance_with_t_test(ps, pf, explain_example)
# print(result)


In [ ]:
import numpy as np
import math
import random
import re
import os
import sys
import io
import ast
import replicate
from scipy import stats

def compare_total_profits(input_str1, input_str2, previous_solution, secondary_solution, result_prev_fb, result_second_fb):
    pattern = r"Total Profits across all products and periods:\s*(\d+\.\d+)"
    match1 = re.search(pattern, input_str1)
    match2 = re.search(pattern, input_str2)
    profit1 = float(match1.group(1)) if match1 else 0
    profit2 = float(match2.group(1)) if match2 else 0
    if profit1 >= profit2:
        winner_profit = profit1
        winner_input = previous_solution
        winner_fb = result_prev_fb
    else:
        winner_profit = profit2
        winner_input = secondary_solution
        winner_fb = result_second_fb
    return winner_profit, winner_input, winner_fb

def calculate_confidence_interval(profits, confidence=0.95):
    n = len(profits)
    mean = np.mean(profits)
    std_err = stats.sem(profits)
    margin_of_error = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return mean, mean - margin_of_error, mean + margin_of_error

def chance_with_confidence_interval(ps, pf, explain_example, max_executions=10, improvement_threshold=0.01, window_size=3, confidence=0.95):
    previous_solution = ps
    previous_feedback = pf
    counter = 0
    seed = 42
    max_retry_attempts = 3

    recent_profits = []
    best_profit = float('-inf')

    while counter < max_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, at the end of the code don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies (and workforce strategies if such elements appear in the current model) based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned and such strategies exist already, still add at least one more. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX and so on ..... If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function, or for workforce), description strategy, and reason for this: (= which problem(s) detected in the last performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!). Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out at least one additional extension/new strategy compared to the LP model that I've shown you as the current solution.",
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while attempt <= max_retry_attempts:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )

                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout

                exec(py_code_s[0])

                sys.stdout = old_stdout

                break
            except Exception as e:
                print(f'Failed attempt {attempt}: {e}')
                attempt += 1

        if attempt > max_retry_attempts:
            print("Max retry attempts reached. Returning the current winner version.")
            return previous_solution

        py_code_p = extract_python_code_manual(previous_solution)

        execs = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])",
            "Overtime": r"Overtime: (\[.*?\])",
            "Workforce": r"Workforce: (\[.*?\])",
            "Hires": r"Hires: (\[.*?\])",
            "Firings": r"Firings: (\[.*?\])",
            "Profit": r"Profit: (.*?)\n"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")
        overtime_Prev = matrices.get("Overtime")
        workers_Prev = matrices.get("Workforce")
        hired_Prev = matrices.get("Hires")
        fired_Prev = matrices.get("Firings")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")
        overtime_Second = matrices.get("Overtime")
        workers_Second = matrices.get("Workforce")
        hired_Second = matrices.get("Hires")
        fired_Second = matrices.get("Firings")

        result_prev = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[-1]
        result_second = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[-1]

        result_prev_fb = run_extended_simulation(produced_Prev, sale_Prev, overtime_Prev, workers_Prev, hired_Prev, fired_Prev, initial_inventory, seed)[:-1]
        result_second_fb = run_extended_simulation(produced_Second, sale_Second, overtime_Second, workers_Second, hired_Second, fired_Second, initial_inventory, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        if winner_profit > best_profit:
            best_profit = winner_profit

        recent_profits.append(winner_profit)
        if len(recent_profits) > window_size:
            recent_profits.pop(0)

        # Calculate confidence interval
        mean_profit, lower_bound, upper_bound = calculate_confidence_interval(recent_profits, confidence)
        print(f"Confidence Interval: [{lower_bound}, {upper_bound}]")

        if (upper_bound - lower_bound) / mean_profit < improvement_threshold:
            print(f"Confidence interval indicates no substantial improvement. Stopping early.")
            break

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return winner_input

# Sample usage
# ps = "Initial solution string"
# pf = "Initial feedback string"
# explain_example = "Explain example string"
# result = chance_with_confidence_interval(ps, pf, explain_example)
# print(result)


In [ ]:
# Define a function to run parallel tasks, before, the setting was 5/3   max_executions=10
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=5, max_executions=10):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, max_executions=10) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

In [ ]:
collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)



Here is the modified code with new strategies for storage, production, and sales:
```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [ ]:
collect_array

['\n\nHere is the modified code with new strategies for storage, production, and sales:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 

In [ ]:
collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)



Here is the modified code with new strategies for storage, production, and sales:
```python
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Define the function
def optimize_production():
    # Input data
    time_horizon = 6
    products_total = 4
    workstations_total = 4
    ressources_total = 4

    profit = [132.0, 813.0, 225.0, 131.0]
    holding_costs = [13.0, 18.0, 15.0, 13.0]

    min_sales = [
        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],
        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],
        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],
        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],
    ]

    max_demand = [
        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],
        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],
        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],
        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],
    ]

    capacity = [
        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],
        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],
        [203.0, 405.0, 2107

In [ ]:
collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)

In [ ]:
collect_array

['\n\nHere is a revised version of the code with new strategies for storage, production, sales, and workforce:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0,

Collecting the Strategies

In [ ]:
import pandas as pd
import re
from google.colab import files

In [ ]:
import re
import pandas as pd

# Initialize an empty list to store the extracted strategies
all_strategies = []

# Define the regular expression pattern to find the strategy table
table_pattern = re.compile(r"# Table: New Strategies\n# \| Strategy \| Description \| Reason for Implementation \|\n# \| --- \| --- \| --- \|\n((# \| [^|]+\|[^|]+\|[^|]+ \|\n)+)")

# Iterate over each element in collect_array
for text in collect_array:
    matches = table_pattern.findall(text)

    # If matches are found, extract the data
    for match in matches:
        table_data = match[0].strip().split('\n')  # Extract the first element of the tuple
        for row in table_data:
            parts = row.split('|')[1:-1]
            all_strategies.append([part.strip() for part in parts])

# Create a DataFrame if there are any strategies found
if all_strategies:
    df = pd.DataFrame(all_strategies, columns=["Strategy", "Description", "Reason for Implementation"])
    print(df)
else:
    print("No strategy table found in the text.")

# Proceed with the next steps, like creating the JSONL file
fine_tuning_data = df.apply(lambda row: {
    "text": f"### Human: {row['Reason for Implementation']} ### Assistant: {row['Strategy']}: {row['Description']}"
}, axis=1)

jsonl_data = fine_tuning_data.to_json(orient='records', lines=True)

# Write JSONL data to a file
file_name = 'fine_tuning_data.jsonl'
with open(file_name, 'w') as f:
    f.write(jsonl_data)

# Download the file to your local machine
from google.colab import files
files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## ** END**

In [ ]:
import concurrent.futures
import replicate
import time
import sys
import io
import re
import ast

# Define the chance function
def chance(ps, pf, explain_example, number_of_executions=5):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42

    while counter < number_of_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, in the end, don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX, Stock: XXX. If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
            # "max_tokens": 2500  # Set the max_tokens parameter for output max_new_tokens
            "max_new_tokens": 2500
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        attempt = 1
        while True:
            try:
                prediction = client.run(
                    "meta/meta-llama-3-70b-instruct",
                    input=input
                )
                cache_prediction = prediction
                secondary_solution = "".join(prediction)
                print(secondary_solution)

                py_code_s = extract_python_code_manual(secondary_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout
                exec(py_code_s[0])
                sys.stdout = old_stdout
                execs = new_stdout.getvalue()

                py_code_p = extract_python_code_manual(previous_solution)

                old_stdout = sys.stdout
                new_stdout = io.StringIO()
                sys.stdout = new_stdout
                exec(py_code_p[0])
                sys.stdout = old_stdout
                execp = new_stdout.getvalue()

                break
            except Exception as e:
                print(f'Failed attempt {attempt}!')
                attempt += 1

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")

        result_prev = run_simulation(produced_Prev, sale_Prev, seed)[-1]
        result_second = run_simulation(produced_Second, sale_Second, seed)[-1]

        result_prev_fb = run_simulation(produced_Prev, sale_Prev, seed)[:-1]
        result_second_fb = run_simulation(produced_Second, sale_Second, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return previous_solution, previous_feedback

# Define a function to run parallel tasks
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=5, number_of_executions=2):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, number_of_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array


In [ ]:
!pip install concurrent

In [ ]:
def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=2, number_of_executions=2):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, number_of_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)

NameError: name 'concurrent' is not defined

In [ ]:
collect_array

['```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\ndef advanced_supply_chain_optimization():\n    # Input Data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n

In [ ]:
patterns = {
    "Produced": r"Produced: (\[\[.*?\]\])",
    "Sale": r"Sale: (\[\[.*?\]\])",
    "Stock": r"Stock: (\[\[.*?\]\])",
    "Overtime": r"Overtime: (\[.*?\])",
    "Workforce": r"Workforce: (\[.*?\])",
    "Hires": r"Hires: (\[.*?\])",
    "Firings": r"Firings: (\[.*?\])",
    "Profit": r"Profit: (.*?)\n"
}

# Dictionary to hold the extracted matrices
matrices = {}

# Extract and evaluate each matrix
for key, pattern in patterns.items():
    match = re.search(pattern, execp, re.DOTALL)
    if match:
        matrices[key] = ast.literal_eval(match.group(1))

# Access the matrices
produced_Prev = matrices.get("Produced")
sale_Prev = matrices.get("Sale")
stock_Prev = matrices.get("Stock")

overtime_Prev = matrices.get("Overtime")
workers_Prev = matrices.get("Workforce")
hired_Prev = matrices.get("Hires")
fired_Prev = matrices.get("Firings")

In [ ]:
collect_array

NameError: name 'collect_array' is not defined

In [ ]:
import concurrent

def chance(ps, pf, explain_example, number_of_executions=5):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42

    while counter < number_of_executions:
        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, in the end, don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX, Stock: XXX. If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
            "max_tokens": 2000  # Set the max_tokens parameter for output
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        with lock:
            prediction = client.run(
                "meta/meta-llama-3-70b-instruct",
                input=input
            )
            time.sleep(1)  # Adjust sleep to comply with API rate limits

        secondary_solution = "".join(prediction)
        print(secondary_solution)

        py_code_p = extract_python_code_manual(previous_solution)
        py_code_s = extract_python_code_manual(secondary_solution)

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()

        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_s[0])
        sys.stdout = old_stdout
        execs = new_stdout.getvalue()

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")

        result_prev = run_simulation(produced_Prev, sale_Prev, seed)[-1]
        result_second = run_simulation(produced_Second, sale_Second, seed)[-1]

        result_prev_fb = run_simulation(produced_Prev, sale_Prev, seed)[:-1]
        result_second_fb = run_simulation(produced_Second, sale_Second, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1}")

        counter += 1
        seed += 1

    return previous_solution, previous_feedback

def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=2, number_of_executions=2):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, number_of_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

# Example usage:
previous_solution_bu = "..."  # Define your initial previous_solution
previous_feedback_bu = "..."  # Define your initial previous_feedback
explain_example = "..."  # Define your explanation example

collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)

In [ ]:
import concurrent.futures
import replicate
import sys
import io
import re
import ast
import time
from threading import Lock

lock = Lock()


def chance(ps, pf, explain_example, number_of_executions=5):
    previous_solution = ps
    previous_feedback = pf

    counter = 0
    seed = 42

    while counter < number_of_executions:
        print(f"Execution {counter + 1}")

        input = {
            "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, in the end, don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX, Stock: XXX. If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
            "max_tokens": 2000  # Set the max_tokens parameter for output
        }

        client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

        with lock:
            print("Requesting prediction...")
            prediction = client.run(
                "meta/meta-llama-3-70b-instruct",
                input=input
            )
            time.sleep(1)  # Adjust sleep to comply with API rate limits
            print("Prediction received.")

        secondary_solution = "".join(prediction)
        print("Secondary solution received.")

        py_code_p = extract_python_code_manual(previous_solution)
        py_code_s = extract_python_code_manual(secondary_solution)

        print("Executing previous solution...")
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_p[0])
        sys.stdout = old_stdout
        execp = new_stdout.getvalue()
        print("Previous solution executed.")

        print("Executing secondary solution...")
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout
        exec(py_code_s[0])
        sys.stdout = old_stdout
        execs = new_stdout.getvalue()
        print("Secondary solution executed.")

        patterns = {
            "Produced": r"Produced: (\[\[.*?\]\])",
            "Sale": r"Sale: (\[\[.*?\]\])",
            "Stock": r"Stock: (\[\[.*?\]\])"
        }

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execp, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Prev = matrices.get("Produced")
        sale_Prev = matrices.get("Sale")
        stock_Prev = matrices.get("Stock")

        matrices = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, execs, re.DOTALL)
            if match:
                matrices[key] = ast.literal_eval(match.group(1))

        produced_Second = matrices.get("Produced")
        sale_Second = matrices.get("Sale")
        stock_Second = matrices.get("Stock")

        print("Running simulations...")
        result_prev = run_simulation(produced_Prev, sale_Prev, seed)[-1]
        result_second = run_simulation(produced_Second, sale_Second, seed)[-1]

        result_prev_fb = run_simulation(produced_Prev, sale_Prev, seed)[:-1]
        result_second_fb = run_simulation(produced_Second, sale_Second, seed)[:-1]

        winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
        print("Higher Total Profit:", winner_profit)
        print("Winner Input:", winner_input)
        print("Winner Feedback:", winner_fb)

        previous_solution = winner_input
        previous_feedback = winner_fb

        print(f"Execution {counter + 1} completed.")

        counter += 1
        seed += 1

    return previous_solution, previous_feedback

def run_parallel_tasks(ps, pf, explain_example, number_of_iterations=2, number_of_executions=2):
    collect_array = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(chance, ps, pf, explain_example, number_of_executions) for _ in range(number_of_iterations)]
        for future in concurrent.futures.as_completed(futures):
            try:
                collect_array.append(future.result())
            except Exception as e:
                print(f"An error occurred: {e}")
    return collect_array

collect_array = run_parallel_tasks(previous_solution_bu, previous_feedback_bu, explain_example)
print(collect_array)


NameError: name 'previous_solution_bu' is not defined

In [ ]:
collect_array

[]

In [ ]:
collect_array

[]

In [ ]:
counter = 0
seed = 42

# Set the number of times you want to execute the code
number_of_executions = 3

while counter < number_of_executions:

    input = {
    "prompt": f"Print out the Python code for a production planning problem. {previous_solution} is the current solution. Also use the same format/notation how the chunks are written differentiated from the rest of the text (```python at the beginning of the code, ``` at the end of the code). In the beginning of the code, don't forget the python in ```python, in the end, don't forget the ```!!!!. Create a solution that is similar to the existing one but not the same, therefore, alter the optimization function or add additional constraints. Don't forget to always include the input data and to run the function after defining it. This is the performance feedback of the current solution obtained by simulation: {previous_feedback}. To do this, find new storage strategies, production, and sales strategies based on the existing feedback of the current solution. Concentrate on these strategies that I just mentioned. Also do this in a refined manner: Sometimes, apply certain policies to only certain products and certain time periods. Do not alter the real life input data, but change the optimization function and/or add constraints or extend the existing constraints! Write the code in a way that when running the code, the decision variables will be outputted as (multidimensional) arrays. Also return the profit earned. Do not omit any piece of code or input data, because I immediately want to run the code you outputted just as it is. For the printout of the decision variables amount produced and sales and stock use this exact terminology: Produced: XXX, Sale: XXX, Stock: XXX. If you use packages other than pulp, also write import XXX into the code. When you write the comments like # New strategy for storage (resp. for production, for sales, for objective function): [description strategy], reason for this: [what problem detected in the performance feedback the newly added strategy is working against. Explicitly name the problem spotted in the simulation feedback!]. Add these information (description and the 'why' at last behind the code in the form of a table), the shape must be exactly like in the given example: {explain_example}. These are just examples and should show the data structure! Try out only one additional extension compared to the LP model that I've shown you.",
    "max_tokens": 2500  # Set the max_tokens parameter for output
    }

    # Make the prediction using the correct method to pass the token
    client = replicate.Client(api_token=os.getenv("REPLICATE_API_TOKEN"))

    # Make the prediction
    prediction = client.run(
        "meta/meta-llama-3-70b-instruct",
        input=input
    )

    # Print the output
    secondary_solution = "".join(prediction)
    print(secondary_solution)

    # Extracting the python code
    py_code_p = extract_python_code_manual(previous_solution)
    py_code_s = extract_python_code_manual(secondary_solution)

    # Redirect stdout to capture prints
    old_stdout = sys.stdout  # Save the old state
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    # Execute the code
    exec(py_code_p[0])
    # Restore stdout to its original state
    sys.stdout = old_stdout
    # Get the output and work with it
    execp = new_stdout.getvalue()

    # Redirect stdout to capture prints
    old_stdout = sys.stdout  # Save the old state
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    # Execute the code
    exec(py_code_s[0])
    # Restore stdout to its original state
    sys.stdout = old_stdout
    # Get the output and work with it
    execs = new_stdout.getvalue()

    # Define regex patterns for each matrix
    patterns = {
        "Produced": r"Produced: (\[\[.*?\]\])",
        "Sale": r"Sale: (\[\[.*?\]\])",
        "Stock": r"Stock: (\[\[.*?\]\])"
    }

    # Dictionary to hold the extracted matrices
    matrices = {}

    # Extract and evaluate each matrix
    for key, pattern in patterns.items():
        match = re.search(pattern, execp, re.DOTALL)
        if match:
            matrices[key] = ast.literal_eval(match.group(1))

    # Access the matrices
    produced_Prev = matrices.get("Produced")
    sale_Prev = matrices.get("Sale")
    stock_Prev = matrices.get("Stock")


    # Define regex patterns for each matrix
    patterns = {
        "Produced": r"Produced: (\[\[.*?\]\])",
        "Sale": r"Sale: (\[\[.*?\]\])",
        "Stock": r"Stock: (\[\[.*?\]\])"
    }

    # Dictionary to hold the extracted matrices
    matrices = {}

    # Extract and evaluate each matrix
    for key, pattern in patterns.items():
        match = re.search(pattern, execs, re.DOTALL)
        if match:
            matrices[key] = ast.literal_eval(match.group(1))

    # Access the matrices
    produced_Second = matrices.get("Produced")
    sale_Second = matrices.get("Sale")
    stock_Second = matrices.get("Stock")

    result_prev = run_simulation(produced_Prev, sale_Prev, seed)[-1]
    result_second = run_simulation(produced_Second, sale_Second, seed)[-1]

    result_prev_fb = run_simulation(produced_Prev, sale_Prev, seed)[:-1]
    result_second_fb = run_simulation(produced_Second, sale_Second, seed)[:-1]

    winner_profit, winner_input, winner_fb = compare_total_profits(result_prev, result_second, previous_solution, secondary_solution, result_prev_fb, result_second_fb)
    print("Higher Total Profit:", winner_profit)
    print("Winner Input:", winner_input)
    print("Winner Feedback:", winner_fb)

    previous_solution = winner_input
    previous_feedback = winner_fb

        # Example operation
    print(f"Execution {counter + 1}")

    # Increment the counter at the end of each loop iteration
    counter += 1
    seed = seed+1

In [ ]:
import re
import pandas as pd

# Initialize an empty list to store the extracted strategies
all_strategies = []

# Define the regular expression pattern to find the strategy table
table_pattern = re.compile(r"# Table: New Strategies\n# \| Strategy \| Description \| Reason for Implementation \|\n# \| --- \| --- \| --- \|\n((# \| [^|]+\|[^|]+\|[^|]+ \|\n)+)")

# Iterate over each element in collect_array
for text in collect_array:
    match = table_pattern.search(text)

    # If a match is found, extract the data
    if match:
        table_data = match.group(1).strip().split('\n')
        for row in table_data:
            parts = row.split('|')[1:-1]
            all_strategies.append([part.strip() for part in parts])

# Create a DataFrame if there are any strategies found
if all_strategies:
    df = pd.DataFrame(all_strategies, columns=["Strategy", "Description", "Reason for Implementation"])
    print(df)
else:
    print("No strategy table found in the text.")

# Proceed with the next steps, like creating the JSONL file
fine_tuning_data = df.apply(lambda row: {
    "text": f"### Human: {row['Reason for Implementation']} ### Assistant: {row['Strategy']}: {row['Description']}"
}, axis=1)

jsonl_data = fine_tuning_data.to_json(orient='records', lines=True)

# Write JSONL data to a file
file_name = 'fine_tuning_data.jsonl'
with open(file_name, 'w') as f:
    f.write(jsonl_data)

# Download the file to your local machine
from google.colab import files
files.download(file_name)


In [ ]:
# Use regular expressions to find the strategy table in the text
table_pattern = re.compile(r"# Table: New Strategies\n# \| Strategy \| Description \| Reason for Implementation \|\n# \| --- \| --- \| --- \|\n# \| ([^|]+) \| ([^|]+) \| ([^|]+) \|")
match = table_pattern.search(collect_array[1])

# If a match is found, extract the data
if match:
    strategy = match.group(1).strip()
    description = match.group(2).strip()
    reason = match.group(3).strip()

    # Create a DataFrame
    df = pd.DataFrame([{"Strategy": strategy, "Description": description, "Reason for Implementation": reason}])

    # Display the DataFrame
    print(df)
else:
    print("No strategy table found in the text.")

In [ ]:
# Use regular expressions to find the strategy table in the text
table_pattern = re.compile(r"# Table: New Strategies\n# \| Strategy \| Description \| Reason for Implementation \|\n# \| --- \| --- \| --- \|\n((# \| [^|]+\|[^|]+\|[^|]+ \|\n)+)")
match = table_pattern.search(collect_array[1])

# If a match is found, extract the data
if match:
    table_data = match.group(1).strip().split('\n')
    strategies = []
    for row in table_data:
        parts = row.split('|')[1:-1]
        strategies.append([part.strip() for part in parts])

    # Create a DataFrame
    df = pd.DataFrame(strategies, columns=["Strategy", "Description", "Reason for Implementation"])

    # Display the DataFrame
    print(df)
else:
    print("No strategy table found in the text.")

In [ ]:
df = pd.DataFrame(df)

# Creating the JSONL dataset for fine-tuning
fine_tuning_data = df.apply(lambda row: {
    "text": f"### Human: {row['Reason for Implementation']} ### Assistant: {row['Strategy']}: {row['Description']}"
}, axis=1)

jsonl_data = fine_tuning_data.to_json(orient='records', lines=True)

jsonl_data

In [ ]:
# Write JSONL data to a file
with open('fine_tuning_data.jsonl', 'w') as f:
    f.write(jsonl_data)

# Download the file to your local machine
files.download('fine_tuning_data.jsonl')

In [ ]:
'\n\nHere is the revised code with new strategies for storage, production, sales, and workforce:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # New data required for the function\n    storage_capacity = [100, 150, 200, 250]  # New strategy for storage\n    production_limit = [500, 600, 700, 800]  # New strategy for production\n    sales_discount = [0.05, 0.1, 0.15, 0.2]  # New strategy for sales\n    workforce_training = [10, 20, 30, 40]  # New strategy for workforce\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \\\n                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # New constraints for storage, production, sales, and workforce\n    for i in products:\n        for t in periods:\n            # Storage capacity constraint\n            problem += Stock[i][t] <= storage_capacity[i], f"Storage_Capacity_{i}_{t}"\n            # Production limit constraint\n            problem += Produced[i][t] <= production_limit[i], f"Production_Limit_{i}_{t}"\n            # Sales discount constraint\n            problem += Sale[i][t] >= profit[i] * (1 - sales_discount[i]), f"Sales_Discount_{i}_{t}"\n            # Workforce training constraint\n            problem += Workforce[t] >= workforce_training[i], f"Workforce_Training_{i}_{t}"\n\n    # Additional strategy: Prioritize production of high-demand products in early periods\n    for i in products:\n        for t in range(2):\n            problem += Produced[i][t] >= 0.5 * max_demand[i][t], f"Prioritize_High_Demand_{i}_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Storage Capacity | Limit storage capacity to 100, 150, 200, and 250 units for each product | To prevent overstocking and reduce holding costs |\n# | Production Limit | Limit production to 500, 600, 700, and 800 units for each product | To prevent overproduction and reduce waste |\n# | Sales Discount | Offer a discount of 5%, 10%, 15%, and 20% on sales for each product | To increase sales and revenue |\n# | Workforce Training | Require a minimum of 10, 20, 30, and 40 hours of training for each product | To improve workforce productivity and efficiency |\n# | Prioritize High-Demand Products | Prioritize production of high-demand products in early periods | To meet high demand and reduce missed sales |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added a new strategy to prioritize production of high-demand products in early periods, which should help reduce missed sales and increase revenue. I also refined the existing',
 '\n\nHere is a revised version of the code that incorporates new strategies for storage, production, and sales:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    resources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n    safety_stock_level = 10.0  # New strategy for storage\n    advance_production_quantity = 10.0  # New strategy for production\n    discount_rate = 0.1  # New strategy for sales\n    flexible_production_rate = 0.2  # New strategy for production\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            # Safety stock constraint\n            problem += Stock[i][t] >= safety_stock_level, f"Safety_Stock_{i}_{t}"\n            # Advance production constraint\n            if t < time_horizon - 1:\n                problem += Produced[i][t] >= advance_production_quantity, f"Advance_Production_{i}_{t}"\n            # Discount constraint\n            if t > 0 and t % 2 == 0:\n                problem += Sale[i][t] >= Sale[i][t-1] * (1 - discount_rate), f"Discount_{i}_{t}"\n            # Flexible production constraint\n            if t % 2 == 0:\n                problem += Produced[i][t] >= flexible_production_rate * max_demand[i][t], f"Flexible_Production_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \\\n                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n    \n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Safety Stock | Implement a safety stock level to avoid stockouts | Avoid stockouts and ensure a certain level of inventory |\n# | Advance Production | Produce a certain quantity of each product in advance to meet future demand | Meet future demand and reduce the risk of stockouts |\n# | Discounts | Offer discounts for certain products during certain periods to increase sales | Increase sales and revenue |\n# | Flexible Production | Produce products at a flexible rate to adapt to changing demand | Adapt to changing demand and reduce production costs |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added a new strategy for production, called "Flexible Production", which allows for a flexible production rate to adapt to changing demand. I also added a new constraint to ensure that the production rate is above a certain level during certain periods.\n\nNote that I used the same notation and format as the original code, and added the new strategy and constraint in a way that is consistent with the existing code.

In [ ]:
 '\n\nHere is the revised code with new strategies for storage, production, sales, and workforce:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    ressources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n\n    # New data required for the function\n    storage_capacity = [100, 150, 200, 250]  # New strategy for storage\n    production_limit = [500, 600, 700, 800]  # New strategy for production\n    sales_discount = [0.05, 0.1, 0.15, 0.2]  # New strategy for sales\n    workforce_training = [10, 20, 30, 40]  # New strategy for workforce\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \\\n                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # New constraints for storage, production, sales, and workforce\n    for i in products:\n        for t in periods:\n            # Storage capacity constraint\n            problem += Stock[i][t] <= storage_capacity[i], f"Storage_Capacity_{i}_{t}"\n            # Production limit constraint\n            problem += Produced[i][t] <= production_limit[i], f"Production_Limit_{i}_{t}"\n            # Sales discount constraint\n            problem += Sale[i][t] >= profit[i] * (1 - sales_discount[i]), f"Sales_Discount_{i}_{t}"\n            # Workforce training constraint\n            problem += Workforce[t] >= workforce_training[i], f"Workforce_Training_{i}_{t}"\n\n    # Additional strategy: Prioritize production of high-demand products in early periods\n    for i in products:\n        for t in range(2):\n            problem += Produced[i][t] >= 0.5 * max_demand[i][t], f"Prioritize_High_Demand_{i}_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n\n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Storage Capacity | Limit storage capacity to 100, 150, 200, and 250 units for each product | To prevent overstocking and reduce holding costs |\n# | Production Limit | Limit production to 500, 600, 700, and 800 units for each product | To prevent overproduction and reduce waste |\n# | Sales Discount | Offer a discount of 5%, 10%, 15%, and 20% on sales for each product | To increase sales and revenue |\n# | Workforce Training | Require a minimum of 10, 20, 30, and 40 hours of training for each product | To improve workforce productivity and efficiency |\n# | Prioritize High-Demand Products | Prioritize production of high-demand products in early periods | To meet high demand and reduce missed sales |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added a new strategy to prioritize production of high-demand products in early periods, which should help reduce missed sales and increase revenue. I also refined the existing',
 '\n\nHere is a revised version of the code that incorporates new strategies for storage, production, and sales:\n```python\nfrom pulp import LpMaximize, LpProblem, LpVariable, lpSum\n\n# Define the function\ndef optimize_production():\n    # Input data\n    time_horizon = 6\n    products_total = 4\n    workstations_total = 4\n    resources_total = 4\n\n    profit = [132.0, 813.0, 225.0, 131.0]\n    holding_costs = [13.0, 18.0, 15.0, 13.0]\n\n    min_sales = [\n        [71.0, 28.0, 110.0, 78.0, 45.0, 90.0],\n        [5.0, 21.0, 3.0, 211.0, 55.0, 9.0],\n        [23.0, 105.0, 27.0, 75.0, 95.0, 43.0],\n        [20.0, 36.0, 9.0, 29.0, 30.0, 20.0],\n    ]\n\n    max_demand = [\n        [371.0, 228.0, 111.0, 478.0, 245.0, 190.0],\n        [425.0, 221.0, 381.0, 211.0, 155.0, 90.0],\n        [203.0, 415.0, 217.0, 475.0, 95.0, 143.0],\n        [200.0, 316.0, 479.0, 259.0, 130.0, 203.0],\n    ]\n\n    capacity = [\n        [3071.0, 228.0, 1011.0, 4708.0, 2405.0, 1000.0],\n        [4205.0, 2201.0, 381.0, 2101.0, 105.0, 900.0],\n        [203.0, 405.0, 2107.0, 4075.0, 905.0, 1403.0],\n        [200.0, 3016.0, 4709.0, 2059.0, 130.0, 2003.0],\n    ]\n\n    production_time = [\n        [1.0, 2.0, 1.0, 4.0, 2.0, 9.0],\n        [2.0, 1.0, 3.0, 1.0, 5.0, 9.0],\n        [2.0, 1.0, 2.0, 4.0, 5.0, 1.0],\n        [2.0, 6.0, 4.0, 2.0, 1.0, 2.0],\n    ]\n\n    initial_inventory = [41.0, 58.0, 31.0, 38.0]\n    yield_loss = [\n        [0.9, 0.7, 0.9, 0.8],\n        [0.8, 0.6, 0.95, 0.8],\n        [0.9, 0.75, 0.9, 0.78],\n        [0.95, 0.87, 0.97, 0.98],\n    ]\n    capacity_reduction_factor = 0.9\n\n    # New data required for the function\n    worker_hours_per_product = 2.0  # Assumption\n    hourly_wage = 20.0  # Assumption\n    hourly_wage_overtime = 35.0  # Assumption\n    hiring_cost = 10.0  # Assumption\n    layoff_cost = 5.0  # Assumption\n    initial_workers = 10  # Assumption\n    safety_stock_level = 10.0  # New strategy for storage\n    advance_production_quantity = 10.0  # New strategy for production\n    discount_rate = 0.1  # New strategy for sales\n    flexible_production_rate = 0.2  # New strategy for production\n\n    # Create the problem\n    problem = LpProblem("Advanced_Supply_Chain_Optimization", LpMaximize)\n\n    # Index ranges for products, workstations, and periods\n    products = range(products_total)\n    workstations = range(workstations_total)\n    periods = range(time_horizon)\n\n    # Decision Variables\n    Produced = [[LpVariable(f"Produced_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Sale = [[LpVariable(f"Sale_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Stock = [[LpVariable(f"Stock_{i}_{t}", lowBound=0, cat=\'Continuous\') for t in periods] for i in products]\n    Workforce = [LpVariable(f"Workforce_{t}", lowBound=0) for t in periods]\n    Hires = [LpVariable(f"Hires_{t}", lowBound=0) for t in periods]\n    Firings = [LpVariable(f"Firings_{t}", lowBound=0) for t in periods]\n    Overtime = [LpVariable(f"Overtime_{t}", lowBound=0) for t in periods]\n\n    # Objective Function\n    profit_term = lpSum(profit[i] * Sale[i][t] for i in products for t in periods)\n    holding_cost_term = lpSum(holding_costs[i] * Stock[i][t] for i in products for t in periods)\n    worker_cost_term = lpSum(hourly_wage * Workforce[t] for t in periods)\n    overtime_cost_term = lpSum(hourly_wage_overtime * Overtime[t] for t in periods)\n    hires_cost_term = lpSum(hiring_cost * Hires[t] for t in periods)\n    firings_cost_term = lpSum(layoff_cost * Firings[t] for t in periods)\n    problem += profit_term - holding_cost_term - worker_cost_term - overtime_cost_term - hires_cost_term - firings_cost_term\n\n    # Constraints\n    for i in products:\n        for t in periods:\n            # Sales constraints\n            problem += Sale[i][t] >= min_sales[i][t], f"Min_Sales_{i}_{t}"\n            problem += Sale[i][t] <= max_demand[i][t], f"Max_Demand_{i}_{t}"\n            # Inventory balance constraint\n            if t == 0:\n                problem += Stock[i][t] == initial_inventory[i] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            else:\n                problem += Stock[i][t] == Stock[i][t-1] + Produced[i][t] - Sale[i][t], f"Stock_Balance_{i}_{t}"\n            # Safety stock constraint\n            problem += Stock[i][t] >= safety_stock_level, f"Safety_Stock_{i}_{t}"\n            # Advance production constraint\n            if t < time_horizon - 1:\n                problem += Produced[i][t] >= advance_production_quantity, f"Advance_Production_{i}_{t}"\n            # Discount constraint\n            if t > 0 and t % 2 == 0:\n                problem += Sale[i][t] >= Sale[i][t-1] * (1 - discount_rate), f"Discount_{i}_{t}"\n            # Flexible production constraint\n            if t % 2 == 0:\n                problem += Produced[i][t] >= flexible_production_rate * max_demand[i][t], f"Flexible_Production_{i}_{t}"\n\n    for j in workstations:\n        for t in periods:\n            # Adjusted capacity constraint\n            problem += lpSum(production_time[i][j] * Produced[i][t] / yield_loss[i][j] for i in products) <= \\\n                       capacity[j][t] * capacity_reduction_factor, f"Adjusted_Capacity_{j}_{t}"\n\n    for t in periods:\n        # Workforce constraints\n        problem += lpSum(Produced[i][t] for i in products) * worker_hours_per_product <= Workforce[t] + Overtime[t], f"Workforce_Requirement_{t}"\n        if t == 0:\n            problem += Workforce[t] == initial_workers + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n        else:\n            problem += Workforce[t] == Workforce[t-1] + Hires[t] - Firings[t], f"Workforce_Balance_{t}"\n\n    # Solve the problem\n    problem.solve()\n\n    # Output the results\n    print_result = lambda s, v: [f"{s}: {[[v[i][t].varValue for t in periods] for i in products]}" if isinstance(v[0], list) else f"{s}: {[v[t].varValue for t in periods]}" for v in [v]]\n\n    # Extract the optimization results\n    results = {\n        "Produced": [[Produced[i][t].varValue for t in periods] for i in products],\n        "Sale": [[Sale[i][t].varValue for t in periods] for i in products],\n        "Stock": [[Stock[i][t].varValue for t in periods] for i in products],\n        "Workforce": [Workforce[t].varValue for t in periods],\n        "Hires": [Hires[t].varValue for t in periods],\n        "Firings": [Firings[t].varValue for t in periods],\n        "Overtime": [Overtime[t].varValue for t in periods],\n        "Profit": pulp.value(problem.objective)\n    }\n    \n    for key, value in results.items():\n        print(f"{key}: {value}")\n\n    return results\n\n# Table: New Strategies\n# | Strategy | Description | Reason for Implementation |\n# | --- | --- | --- |\n# | Safety Stock | Implement a safety stock level to avoid stockouts | Avoid stockouts and ensure a certain level of inventory |\n# | Advance Production | Produce a certain quantity of each product in advance to meet future demand | Meet future demand and reduce the risk of stockouts |\n# | Discounts | Offer discounts for certain products during certain periods to increase sales | Increase sales and revenue |\n# | Flexible Production | Produce products at a flexible rate to adapt to changing demand | Adapt to changing demand and reduce production costs |\n\n# Call the function to optimize production\noptimize_production()\n```\nI added a new strategy for production, called "Flexible Production", which allows for a flexible production rate to adapt to changing demand. I also added a new constraint to ensure that the production rate is above a certain level during certain periods.\n\nNote that I used the same notation and format as the original code, and added the new strategy and constraint in a way that is consistent with the existing code.'